<h1 id="tocheading">TABLE OF CONTENTS</h1>
<div id="toc"></div>

**Updates to the table of contents are periodic, but run the cell below to first start or force an update.**

In [ ]:
macro javascript_str(s) display("text/javascript", s); end

javascript"""
$.getScript('make_table_of_contents.js')
"""

# Utilities for the results database

The database is assumed to be a dictionary, with keys that are strings. One key, `"args"`, should hold a vector of strings identifying the parameters minimized. All other keys should have vectors or matrices that have nruns rows, each row corresponding to a run. They may have multiple columns, corresponding to multiple entries for that run (e.g., there is typically a `"params"` key that has, for each run, the values of all the arguments, i.e., it will have as many columns as `"args"` is long.

Some **required** keys include

* `"args"` a vector of strings, indicating the name of the trained arguments, common across runs
* `"params"`  Float64 matrix, values of the parameters corresponding to args

Some **typical** keys include

* `"tcost"`  Training cost
* `"cost"`   Test cost
* `"files"`  Name of the file in which the run is stored



In [223]:
"""
"""
function selectize(res, selector::String)
    nruns  = size(res["params"],1)
    mykeys = setdiff(keys(res), ["args"])    
    args   = res["args"]
    
    I = zeros(nruns,1)
    for i=1:nruns
        myDict = Dict()
        for k in mykeys
            if k=="params"
                for a=1:length(args)
                    myDict[args[a]] = res[k][i,a]
                end
            else
                myDict[k] = res[k][i]
            end        
        end
        I[i] = replacer(selector, myDict)
    end
    I = find(I .!= 0)
    
    myres = copy(res)
    for k in mykeys
        myres[k] = res[k][I,:]
    end
    
    return myres, I
end

selectize (generic function with 2 methods)

In [235]:
testres = Dict("args"=>["t1", "t2", "p1", "p2"], "params"=>rand(6,4), "cost"=>[1;2;3;4;5;6], "z"=>[6;5;4;3;2;1])
testres["cost"][6]
selectize(testres, "p1  > 0.4")["params"][:,3]

key cost, run 1
key params, run 1
key z, run 1
key cost, run 2
key params, run 2
key z, run 2
key cost, run 3
key params, run 3
key z, run 3
key cost, run 4
key params, run 4
key z, run 4
key cost, run 5
key params, run 5
key z, run 5
key cost, run 6
key params, run 6
key z, run 6
[1.0; 0.0; 1.0; 1.0; 0.0; 1.0]


4-element Array{Float64,1}:
 0.454257
 0.989829
 0.614056
 0.730101

You selected file MiniFarms/farm_C17_Farms025_0165.jld
anti_rule_strength  		0.123764
right_light_excitation		1.8034
sW                  		1.42384
sigma               		-0.349608
opto_strength       		0.49607
const_pro_bias      		0.0434069
dW                  		0.385437
target_period_excitation		-1.19886
hW                  		0.249509
vW                  		-0.417882
pro_rule_strength   		0.0534352
constant_excitation 		-0.7241
You selected file MiniFarms/farm_C17_Farms024_0114.jld
anti_rule_strength  		0.106605
right_light_excitation		1.54474
sW                  		1.57738
sigma               		0.278245
opto_strength       		0.402333
const_pro_bias      		0.0441954
dW                  		0.447085
target_period_excitation		-0.902554
hW                  		0.292386
vW                  		-0.637033
pro_rule_strength   		0.0308962
constant_excitation 		-0.747265


# Optimizing the C17 farms

In [ ]:
#@include_me  optimizing_C17.jl


include("pro_anti.jl")

In [ ]:
#@include_me  optimizing_C17.jl

mypars = Dict(
:init_add               =>          0,
:const_add              =>          0,
:noise                  =>          Any[],
:input                  =>          0,
:start_anti             =>          [-0.5, -0.5, -0.5, -0.5],
:start_pro              =>          [-0.5, -0.5, -0.5, -0.5],
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1.2],
:target_period          =>          0.3,
:target_periods         =>          [0.3],
:post_target_period     =>          0.3,
:post_target_periods    =>          [0.3],
:anti_rule_strength     =>          0.054,
:U_rest                 =>          0,
:theta                  =>          0.05,
:beta                   =>          0.5,
:g_leak                 =>          1,
:nsteps                 =>          301,
:dt                     =>          0.024,
:tau                    =>          0.09,
:right_light_excitation =>          0.49924152955481954,
:opto_strength          =>          0.85,
:opto_periods           =>          String[
                                    "trial_start" "trial_start"; 
                                    "trial_start" "trial_end"; 
                                    "trial_start" "target_start/2"; 
                                    "target_start/2" "target_start"; 
                                    "target_start" "target_end"],
:opto_targets          =>           [
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7],
:theta2 => 0.15,
:theta1 => 0.05,
:sigma => 0.01,
:cbeta => 0.04,
:sW => 0.6416875048295452,
:hW => 0.054701836208134846,
:dW => 0.1267124266934907,
:vW => -1.588850577499782,
:constant_excitation => -0.37242520737694207,
:const_pro_bias => 0.04366897857834884,
:target_period_excitation => 0.15315254453690974,
:right_light_pro_extra => 0,
:pro_rule_strength => 0.05,
:nPro => 200,
:nAnti => 200,
)


extra_pars = Dict(
:plot_list        =>   [], 
:plot_conditions  =>   true,
:verbose          =>   true,
:opto_periods     =>   String["trial_start" "trial_start-0.1"],
:opto_targets     =>   [0.9,  0.7],
:opto_times       =>   ["trial_start", "trial_start-0.1"],       # This one is for run_ntrials
:cbeta            =>   0.001,
:search_range     =>   1.2,
)

search_conditions = Dict(   # :param    default_start   search_box  bound_box
:vW     =>                   [mypars[:vW],                       [-0.5, 0.5],  [-5,   5]], 
:hW     =>                   [mypars[:hW],                       [-0.5, 0.5],  [-5,   5]],
:dW     =>                   [mypars[:dW],                       [-0.5, 0.5],  [-5,   5]],
:sW     =>                   [mypars[:sW],                       [0,    0.5],  [-5,   5]],
:sigma  =>                   [0.11,                              [0.1,  0.2],  [-2,   2]],
:constant_excitation      => [mypars[:constant_excitation],      [-1,     1],  [-30, 30]], 
:target_period_excitation => [mypars[:target_period_excitation], [0.001,0.5],  [-30  30]],
:right_light_excitation   => [mypars[:right_light_excitation],   [0.05, 0.5],  [-30, 30]],
:const_pro_bias           => [mypars[:const_pro_bias],           [-0.5, 0.5],  [-30, 30]],
:opto_strength            => [mypars[:opto_strength],            [0,      1],  [0,    1]],
:pro_rule_strength        => [mypars[:pro_rule_strength],        [0,   0.2,],  [0,   30]],
:anti_rule_strength       => [mypars[:anti_rule_strength],       [0,   0.2,],  [0,   30]],
)


bbox = Dict()
for k in keys(search_conditions)
    bbox = merge(bbox, Dict(k=>search_conditions[k][3]))
end
bbox


In [ ]:
#@include_me  optimizing_C17.jl

README = """
Attempt to optimize the C17 farms, starting from all ending points that had
test costs <= 0.
"""

if ~isnull(tryparse(Int64, ARGS[1])); my_run_number = parse(Int64, ARGS[1]); 
else                                  my_run_number = 1; 
end
if ~isnull(tryparse(Int64, ARGS[2])); tot_n_runs    = parse(Int64, ARGS[2]); 
else                                  tot_n_runs = 1; 
end

source_dir = "MiniFarms"
optim_dir  = "Optimized_C17_Farms"
cost_threshold = 0   # only work with test costs less than this

if ~isdir(optim_dir); mkdir(optim_dir); end


f = readdir(source_dir)
nloops = 0

while my_run_number + (nloops*tot_n_runs) <= length(f)
    myfile = f[my_run_number + (nloops*tot_n_runs)]
    @printf("\n\n*** %d %s: grabbing file %s ***\n\n", my_run_number, 
        Dates.format(now(), "e, dd u yyyy HH:MM:SS"), myfile)
    mypars, extra_pars, args, seed, test_cost = load(source_dir * "/" * myfile, 
        "mypars", "extra_pars", "args", "pars3", "cost")

    if test_cost <= 0
        mypars[:nPro]  = 1600
        mypars[:nAnti] = 1600

        maxiter1 = 1000;   # for func1, the regular search
        testruns = 10000;  # Number of trials for evaluating the results of the model. 10000 is a good number 

        func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=false, 
            merge(merge(mypars, extra_pars), Dict(params))...)[1]

        try
            pars3, traj3, cost3, cpm_traj3, ftraj3 = bbox_Hessian_keyword_minimization(seed, 
                args, bbox, func1, 
                start_eta = 0.01, tol=1e-12, 
                verbose=true, verbose_every=1, maxiter=maxiter1)

            # evaluate the result with many trials, for accuracy
            cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(testruns, testruns; verbose=false, 
                make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)

            # Write out the results
            myfilename = optim_dir * "/" * myfile

            @printf("\n\n ****** writing to file %s *******\n\n", myfilename)

            # write file
            save(myfilename, Dict("README"=>README, "nPro"=>mypars[:nPro], "nAnti"=>mypars[:nAnti], 
                "mypars"=>mypars, "extra_pars"=>extra_pars, "args"=>args, "seed"=>seed, 
                "pars3"=>pars3, "traj3"=>traj3, "cost3"=>cost3, "cpm_traj3"=>cpm_traj3, "ftraj3"=>ftraj3,
                "cost"=>cost, "cost1s"=>cost1s, "cost2s"=>cost2s,
                "hP"=>hP, "hA"=>hA, "dP"=>dP, "dA"=>dA, "hBP"=>hBP, "hBA"=>hBA))

        catch y
            # Interrupts should not get caught:
            if isa(y, InterruptException); throw(InterruptException()); end

            # Other errors get caught and a warning is issued but then we run again
            @printf("\n\nWhoopsety, unkown error!\n\n");
            @printf("Error was :\n"); print(y); @printf("\n\nTrying new random seed.\n\n")
        end
    end
    
    nloops += 1
end

In [ ]:
G["extra_pars"]

# An attempt at opto after reduced_farm.jl

No pre-search yet, but might add it after

In [ ]:
#@include_me  opto_reduced.jl


include("pro_anti.jl")

In [ ]:
#@include_me  opto_reduced.jl

mypars = Dict(
:init_add               =>          0,
:const_add              =>          0,
:noise                  =>          Any[],
:input                  =>          0,
:start_anti             =>          [-0.5, -0.5, -0.5, -0.5],
:start_pro              =>          [-0.5, -0.5, -0.5, -0.5],
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1.2],
:target_period          =>          0.3,
:target_periods         =>          [0.3],
:post_target_period     =>          0.3,
:post_target_periods    =>          [0.3],
:anti_rule_strength     =>          0.054,
:U_rest                 =>          0,
:theta                  =>          0.05,
:beta                   =>          0.5,
:g_leak                 =>          1,
:nsteps                 =>          301,
:dt                     =>          0.024,
:tau                    =>          0.09,
:right_light_excitation =>          0.49924152955481954,
:opto_strength          =>          0.85,
:opto_periods           =>          String[
                                    "trial_start" "trial_start"; 
                                    "trial_start" "trial_end"; 
                                    "trial_start" "target_start/2"; 
                                    "target_start/2" "target_start"; 
                                    "target_start" "target_end"],
:opto_targets          =>           [
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7],
:theta2 => 0.15,
:theta1 => 0.05,
:sigma => 0.01,
:cbeta => 0.04,
:sW => 0.6416875048295452,
:hW => 0.054701836208134846,
:dW => 0.1267124266934907,
:vW => -1.588850577499782,
:constant_excitation => -0.37242520737694207,
:const_pro_bias => 0.04366897857834884,
:target_period_excitation => 0.15315254453690974,
:right_light_pro_extra => 0,
:pro_rule_strength => 0.05,
:nPro => 200,
:nAnti => 200,
)


extra_pars = Dict(
:plot_list        =>   [], 
:plot_conditions  =>   true,
:verbose          =>   true,
:opto_periods     =>   String["trial_start" "trial_start-0.1"],
:opto_targets     =>   [0.9,  0.7],
:opto_times       =>   ["trial_start", "trial_start-0.1"],       # This one is for run_ntrials
:cbeta            =>   0.001,
:search_range     =>   1.2,
)


search_conditions = Dict(   # :param    default_start   search_box  bound_box
:vW     =>                   [mypars[:vW],                       [-0.5, 0.5],  [-5,   5]], 
:hW     =>                   [mypars[:hW],                       [-0.5, 0.5],  [-5,   5]],
:dW     =>                   [mypars[:dW],                       [-0.5, 0.5],  [-5,   5]],
:sW     =>                   [mypars[:sW],                       [0,    0.5],  [-5,   5]],
:sigma  =>                   [0.11,                              [0.1,  0.2],  [-2,   2]],
:constant_excitation      => [mypars[:constant_excitation],      [-1,     1],  [-30, 30]], 
:target_period_excitation => [mypars[:target_period_excitation], [0.001,0.5],  [-30  30]],
:right_light_excitation   => [mypars[:right_light_excitation],   [0.05, 0.5],  [-30, 30]],
:const_pro_bias           => [mypars[:const_pro_bias],           [-0.5, 0.5],  [-30, 30]],
:opto_strength            => [mypars[:opto_strength],            [0,      1],  [0,    1]],
:pro_rule_strength        => [mypars[:pro_rule_strength],        [0,   0.2,],  [0,   30]],
:anti_rule_strength       => [mypars[:anti_rule_strength],       [0,   0.2,],  [0,   30]],
)



In [ ]:
#@include_me  opto_reduced.jl

extra_pars[:seedrand] = Int64(round(1000*time()))   
srand(extra_pars[:seedrand])
search_range = extra_pars[:search_range]; 

README = """

Farm C17: Like C16, with opto_strength and pro_rule_stregth and anti_rule_strength as 
trainable parameters, but with reasonable bounds on the appropriate parameters,
not no bounds at all.

"""

extra_pars[:opto_periods]  = String[
    "trial_start-0.1"     "trial_start-0.2" ; 
    "target_start-0.4"    "target_start" ; 
    "target_start+0.016"  "target_end" ; 
]

extra_pars[:opto_targets] = [
    0.9   0.7  ;
    0.85  0.5  ;
    0.9   0.7  ;
]

ftraj2 = []; cost2 = [];

if !isdir("../NewFarms"); mkdir("../NewFarms"); end
fbasename = "../NewFarms/farm_C17_"
# If we wanted a unique identifier per processor run the following line would help:
# if ~isnull(tryparse(Int64, ARGS[1])); fbasename = fbasename * ARGS[1] * "_"; end

@printf("\n\n\nStarting with random seed %d\n\n\n", extra_pars[:seedrand])

while true
    
    @printf("\n\n--- new run ---\n\n")
    args = []; seed = []; bbox = Dict()
    for k in keys(search_conditions)
        search_box = search_conditions[k][2]
        args = [args; String(k)]
        # --- search within the indicated search range:
        myseed = search_conditions[k][1] + search_range*(rand()-0.5)*diff(search_box); myseed = myseed[1]
        if myseed > search_box[2]; myseed = search_box[2]; end
        if myseed < search_box[1]; myseed = search_box[1]; end
        seed = [seed ;  myseed]
        # --- No search, just start at the indicated position:
        # seed = [seed ; search_conditions[k][1]]
        # --- search within the full indicated search box
        # seed = [seed ; rand()*diff(search_box) + search_box[1]]
        bbox = merge(bbox, Dict(k => Array{Float64}(search_conditions[k][3])))
    end
    args = Array{String}(args)
    seed = Array{Float64}(seed)


    maxiter1 = 1000;   # for func1, the regular search
    testruns = 10000;  # Number of trials for evaluating the results of the model. 10000 is a good number 


    # Make sure to keep the noise frozen over the search, meaning JJ() needs the seedrand parameter
    func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=false, 
        merge(merge(mypars, extra_pars), Dict(params))...)[1]
    
    try
        pars3, traj3, cost3, cpm_traj3, ftraj3 = bbox_Hessian_keyword_minimization(seed, 
            args, bbox, func1, 
            start_eta = 0.01, tol=1e-12, 
            verbose=true, verbose_every=1, maxiter=maxiter1)
            
        # evaluate the result with many trials, for accuracy
        cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(testruns, testruns; verbose=false, 
            make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)
        
        # Write out the results
        myfilename = next_file(fbasename, 4)
        myfilename = myfilename*".jld"

        @printf("\n\n ****** writing to file %s *******\n\n", myfilename)
        
        # write file
        save(myfilename, Dict("README"=>README, "nPro"=>mypars[:nPro], "nAnti"=>mypars[:nAnti], 
            "mypars"=>mypars, "extra_pars"=>extra_pars, "args"=>args, "seed"=>seed, "bbox"=>bbox, 
            "search_conditions"=>search_conditions,
            "pars3"=>pars3, "traj3"=>traj3, "cost3"=>cost3, "cpm_traj3"=>cpm_traj3, "ftraj3"=>ftraj3,
            "cost"=>cost, "cost1s"=>cost1s, "cost2s"=>cost2s,
            "hP"=>hP, "hA"=>hA, "dP"=>dP, "dA"=>dA, "hBP"=>hBP, "hBA"=>hBA))

    catch y
        # Interrupts should not get caught:
        if isa(y, InterruptException); throw(InterruptException()); end

        # Other errors get caught and a warning is issued but then we run again
        @printf("\n\nWhoopsety, unkown error!\n\n");
        @printf("Error was :\n"); print(y); @printf("\n\nTrying new random seed.\n\n")
    end

    # Change random seed before next iteration so we don't get stuck in one loop
    extra_pars[:seedrand] = extra_pars[:seedrand]+1
end
    



# Analyzing runs from reduced_farm.jl : analyze_reduced_farm.jl

In [ ]:
#@include_me  analyze_reduced_farm.jl

include("pro_anti.jl")

#
#  We're assuming that results of reduced_farm.jl were already generated.
#

In [ ]:
#@include_me  analyze_reduced_farm.jl

"""
hBP, hBA, d = function show_run(run_name, ntrials=100; farmdir="FarmFields", further_pars...)

Given a string representing the filename of a run generated in reduced_farm.jl
(for example, "farm_C3_0027.jld"), tries to load that file from within "FarmFields/"
runs it to plot 30 Pro trials in figure 1 and 30 Anti trials in figure 2,
and indicates in those figures the corresponding number of binarized hits.

Any further parameters override the contents of the farm and are passed on to 
run_ntrials(), so for example you can set rule_and_delay_period=0.2

# RETURNS:

- hBP, hBA   binarized Pro % hit and Anti % hit

- d          the raw dictionary obtained from first loading the file.

"""
function show_run(run_name, ntrials=100; farmdir="FarmFields", further_pars...)

    pygui(true)
    figure(1); clf();
    figure(2); clf();

    if endswith(farmdir, "/"); farmdir=farmdir[1:end-1]; end;
    
    if endswith(run_name, ".jld")
        d = load(farmdir * "/" * run_name)

        extra_pars = d["extra_pars"]
        mypars     = d["mypars"]
        args       = d["args"]
        pars       = d["pars"]
    elseif endswith(run_name, ".mat")                                
        d = matread(farmdir * "/" * run_name)
        
        extra_pars = Dict()
        mypars     = symbol_key_ize(d["model_params"])
        args       = d["args"]
        pars       = d["pars"]
    else
        error("Whoa, I only know how to load .mat and .jld files")
    end

    delete!(extra_pars, :plot_list)   # we want our own plot_list
    delete!(extra_pars, :seedrand)    # don't use the farm's seedrand. (Could change this if wanted)

    # Add or replace alternative period values if needed
    # if rule_and_delay_period != nothing; extra_pars[:rule_and_delay_period] = rule_and_delay_period; end;
    # if target_period         != nothing; extra_pars[:target_period]         = target_period;         end;
    # if post_target_period    != nothing; extra_pars[:post_target_period]    = post_target_period;    end;

    # Run it
    proVs, antiVs = run_ntrials(ntrials, ntrials; plot_list=[1:30;], 
        merge(make_dict(args, pars, merge(mypars, extra_pars)), Dict(further_pars))...)[1:2]

    # Add titles with the binarized hit rates
    hBP = 100*length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
    hBA = 100*length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)

    figure(1); subplot(3,1,1); title(@sprintf("%s: binarized Pro  hits = %.1f %%\n", run_name, hBP))
    figure(2); subplot(3,1,1); title(@sprintf("%s: binarized Anti hits = %.1f %%\n", run_name, hBA))
    
    return hBP, hBA, d
end



In [ ]:
#@include_me analyze_reduced_farm.jl

# IDENTIFY GOOD RUNS FROM FARM

using JLD
I = results = files = 0

farm_id = "C4"; farmdir = "FarmFields"
farm_id = "C6"; farmdir = "../NewFarms"
farm_id = "C11"; farmdir = "../NewFarms"

recompute_me = true; if recompute_me || !isfile("Temp/" * farm_id * "_results.jld")
    results = zeros(0,4)

    files = []
    for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(farmdir * "/"))

        hBP, hBA, dP, dA= load(farmdir * "/" * f, "hBP", "hBA", "dP", "dA")
        results = [results; [hBP hBA dP dA]]; 
        files   = [files ; f]
        if length(files)>71; break; end;
    end
    
    I = find((results[:,1] .> 0.85) .& (results[:,2] .> 0.65) .& (results[:,3].>0.9) .& (results[:,4].>0.9) .&
                ((results[:,1]-results[:,2]) .> 0.1) )

    @save "Temp/" * farm_id * "_results.jld" results files I
else
    I, results, files = load("Temp/" * farm_id *"_results.jld", "I", "results", "files");
end

@printf("%.2f %% good runs out of %d\n", 100*length(I)/length(files), length(files))


# C17_browser.jl -- Analyzing results from C farms

In [81]:


include("pro_anti.jl")

using HDF5

####################################################################
#                                                                  
#   Define some helper functions for loading farms,
#   plotting histograms over the parameters, and managing
#   with GUI interactivity.
#
####################################################################

"""
results = farmload(farm_id; farmdir="../NewFarms", verbose=true, verbose_every=10)

Package and return a summary of results from a number of runs.

# PARAMETERS:

- filename     farm_id  (e.g., "C17"); .jld files containing this pattern will be loaded

# OPTIONAL PARAMETERS:

- farmdir      Directory in which the files are found

- verbose      If true, print a brief progress report every verbose_every files loaded

- verbose_every  Used if verbose==true (see above)

# RETURNS:

A dictionary with the keys "tcost" (vector or training costs), "cost" (vector of test costs), 
"dirs" (vector directories in which files  are found), "files" (vector of filenames, including 
paths), "qu_out" (deprecated for farm C17 and other non-pre-search farms), "params"" (nruns-by-
nparams matrix of paramters), "args" (nparams-long vector of parameter names).

Note that most values are nruns-long vectors but "args" is only nparams-long, because they are
the same names across all the runs.
"""
function farmload(farm_id; farmdir="../NewFarms", verbose=true, verbose_every=50)

    if typeof(farmdir)==String; farmdir=[farmdir]; end
    
    results = Dict(); dirs=[]; files =[]; qs=[]; tcosts=[]; costs=[]; pars=[]; n=0;
    for dd in farmdir
        for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(dd * "/"))
            n += 1
            myfile = dd * "/" * f; 
            fj = jldopen(myfile); 
            if exists(fj, "qu_out"); qu_out = load(myfile, "qu_out"); 
            else;                    qu_out = [-1 -1]; 
            end
            close(fj)

            args, params, traj3, cost = load(myfile, "args", "pars3", "traj3", "cost")

            if     !haskey(results, args);       results["args"] = args;
            elseif !all(results["args"].==args); error("Not all files have same args!"); 
            end

            files = [files ; myfile]; dirs = [dirs ; dd]
            qs = [qs ; qu_out]; tcosts = [tcosts; traj3[2,end]]; costs = [costs; cost]
            if length(pars)==0; pars=params'; else pars = [pars ; params']; end
            if verbose && rem(n, verbose_every)==0
                @printf("%s %g\n", myfile, tcosts[end])
            end
        end
    end

    results["dirs"] = dirs
    results["files"]  = files
    results["qu_out"] = qs
    results["tcost"]  = tcosts
    results["cost"]   = costs
    results["params"] = pars

    return results
end





"""
    Data structure for parameter and cost histograms

`histo_params()` returns one of these objects; used to manage GUI handling.
"""
type histo_data
    names::Array{String}
    values::Array{Float64}
    axisHandles::Array{PyCall.PyObject}
    LineHandles::Array{PyCall.PyObject}
    files::Array{String} 
end




"""
HD = histo_params(args, params, tcosts, costs, files; fignum=1, nbins=10)

Histogram each parameter. params should be nentries-by-length(args) in size.
args should be a vector of strings. tcosts and costs should be nentries in length,
and represent trainig cost, and test cost, respectively. files should be a vector
of filenames.

# OPTIONAL PARAMS

- fignum   The figure in which histograms will be plotted.

- nbins    The number of bins to use in each histogram.

"""
function histo_params(args, params, tcosts, costs, files; fignum=1, nbins=10, linewidth=3)

    pygui(true)
    figure(fignum); clf();
    
    HD = histo_data([], [], [], [], [])

    nparams = size(params,2)
    nrows = ceil(nparams/3)+1

    for i=1:nparams;
        HD.axisHandles = [HD.axisHandles ; subplot(nrows,3,i)]; 
        plt[:hist](params[:,i], nbins)
        title(args[i])
        
        myrow = ceil(i/3)
        if myrow < (nrows+1)/2;     axisHeightChange(0.8, lock="t")
        elseif myrow > (nrows+1)/2; axisHeightChange(0.8, lock="b")
        else                        axisHeightChange(0.8, lock="c")
        end    
    end

    HD.axisHandles = [HD.axisHandles ; subplot(nrows, 2, nrows*2-1)]; axisHeightChange(0.8, lock="b"); 
    axisMove(0, -0.025); plt[:hist](tcosts*1000, nbins); title("training cost*1000")

    HD.axisHandles = [HD.axisHandles ; subplot(nrows, 2, nrows*2)];   axisHeightChange(0.8, lock="b"); 
    axisMove(0, -0.025); plt[:hist](costs*1000, nbins); title("test cost*1000")

    for ax in HD.axisHandles
        axes(ax)
        h = plot([0 0 0 ; 0 0 0], [ylim()[1] ; ylim()[2]]*ones(1,3), visible=false, linewidth=linewidth)
        h[1][:set_color]("m"); h[2][:set_color]("b"); h[3][:set_color]("c")
        HD.LineHandles = [HD.LineHandles ; reshape(h[end:-1:1], 1, 3)]
    end
    
    args   = [args ; ["train cost" ; "test cost"]]
    params = [params tcosts*1000 costs*1000]
    
    HD.names  = args
    HD.values = params
    HD.files  = files
    
    return HD
end


"""
    histo_params(res; threshold=-0.0001, further_params...)

Wrapper that calls the other histo_params method, after first selecting for
only  runs that have a test cost less than threshold.  further_params are passed
on to the other histo_params method.

- threshold             training costs below this value are considered "successful" (red dots), 
                        above it are "unsuccessful" (blue dots)

- cost_choice           String, used to indicate which cost will be used for thresholding. It 
                        must be either "cost", indicating the testing cost, or "tcost", the training cost. 

- fignum                The figure in which histograms will be plotted.

- nbins                 The number of bins to use in each histogram.


"""
function histo_params(res; threshold=-0.0001, cost_choice="cost", further_params...)
    args   = res["args"]
    params = res["params"]
    tcost  = res["tcost"]
    cost   = res["cost"]
    files  = res["files"]
    
    if cost_choice=="cost"
        I = find(cost.<threshold)
    elseif cost_choice=="tcost"
        I = find(tcost.<threshold)
    else
        error("cost_choice MUST be one of \"tcost\" or \"cost\"")
    end
    
    return histo_params(args, params[I,:], tcost[I], cost[I], files[I,:]; Dict(further_params)...)
end


"""
    histo_highlight(filename, HD::histo_data)

Assuming that histo_params was called, and returned the HD that is passed to this function,
this function will put up a vertical bar at the values corresponding to the run indicated
by filename.

Will put up to three bars, blue for the most recent one asked for; green for one call ago
to this function; and magenta for the time the function was called two calls ago. That allows
keeping track of a sequence of requested values.

This function can be used for GUI interactivity-- if a button click selects a run, the
corresponding filename can then be used with this function to highlight that run in the
histograms.

# EXAMPLE:

```jldoctest
res = farmload("C17", farmdir="MiniFarms")

HD = histo_params(res)
I = find(res["cost].<-0.0001)  # to match what histo_params shows

histo_highlight(res["files"][I[1]], HD)
histo_highlight(res["files"][I[2]], HD)
histo_highlight(res["files"][I[3]], HD)
```
"""
function histo_highlight(filename, HD::histo_data)
    idx = find(HD.files .== filename)
    if length(idx)==0; @printf("histo_highlight: Couldn't find filename %s, returning\n", filename); return; end
    
    for i=1:length(HD.names)
        for to=size(HD.LineHandles,2):-1:2
            from=to-1
            HD.LineHandles[i,to][:set_xdata](HD.LineHandles[i,from][:get_xdata]())
            HD.LineHandles[i,to][:set_ydata](HD.LineHandles[i,from][:get_ydata]())
            HD.LineHandles[i,to][:set_visible](HD.LineHandles[i,from][:get_visible]())
        end        
        HD.LineHandles[i,1][:set_xdata]([HD.values[idx,i], HD.values[idx,i]])
        HD.LineHandles[i,1][:set_visible](true)
    end
end


"""
Data structure for PCA scatterplots

`plot_PCA()` returns one of these objects; used to manage GUI handling.
"""
type PCAplot_data
    # --- stuff about the data that is plotted:
    mu::Array{Float64}   # This vector of means is subtracted from a params vector
    sd::Array{Float64}   # Which are then ./ by this sd vector to get the whitened params vector
    V::Array{Float64}    # matrix of principal components, one per column
    C::Array{Float64}    # covariance matrix after whitening
    D::Array{Float64}    # Vector of eigenvalues
    I::Array{Int64}      # index into "successful" runs
    nI::Array{Int64}     # index into "unsuccessful"runs
    Vparams::Array{Float64}  # nruns-by-nparams vector of runs, in PCA space
    files::Array{String} # nruns-long vector of corresponding filenames
    # --- stuff about graphics handling of the plots:
    axisHandles::Array{PyCall.PyObject}  # handles to the plotted axes
    axisPCs::Array{Int64}    # naxes-by-2, indicating x- and y-axes PC # for each axis plot
    dotHandles::Array{PyCall.PyObject} # naxes-by-ndots, extra dots plotted
    BP::PyCall.PyObject # the kbMonitorModule.kb_monitor that will keep track of button presses
    callback::Any   # The function to be called after a button press
end





"""
    PCA_highlight(filename, PC::PCAplot_data)

Assuming that `PCA_plot()` was called, and returned the PC that is passed to this function,
this function will put up a colored dot, across all axes, at the values corresponding to the run 
indicated by filename. [Out of the entries in PC::PCAplot_data, this function uses files, 
Vparams, axHandles, dothandles, and axisPCs]

Will put up to three dots, blue for the most recent one asked for; green for one call ago
to this function; and magenta for the time the function was called two calls ago. That allows
keeping track of a sequence of requested values.

This function can be used for GUI interactivity-- if a button click selects a run, the
corresponding filename can then be used with this function to highlight that run in the
scatterplots.

# EXAMPLE:

```jldoctest
res = farmload("C17", farmdir="MiniFarms")

PC = plot_PCA(res)
I = find(res["tcost].<-0.0002)  # to match what histo_params shows

PCA_highlight(res["files"][I[1]], PC)
PCA_highlight(res["files"][I[2]], PC)
PCA_highlight(res["files"][I[3]], PC)
```
"""
function PCA_highlight(fname, PC::PCAplot_data)
    idx = find(PC.files .== fname)
    if length(idx)==0; @printf("PCA_highlight: Couldn't find filename %s, returning\n", fname); return; end

    Vparams = PC.Vparams[idx,:]

    # Move our non-red dots along:
    if length(PC.dotHandles) > 0            
        # The dot in column X will get the coords of the dot in column X-1:
        for i=1:length(PC.axisHandles)
            for to=size(PC.dotHandles,2):-1:2
                from = to-1
                PC.dotHandles[i,to][:set_xdata](PC.dotHandles[i,from][:get_xdata]())
                PC.dotHandles[i,to][:set_ydata](PC.dotHandles[i,from][:get_ydata]())
                PC.dotHandles[i,to][:set_visible](PC.dotHandles[i,from][:get_visible]())
            end
            # And then the dot in column 1 gets the coords of the red dot closest to the clicked point:
            myX = PC.axisPCs[i,1]; myY = PC.axisPCs[i,2]
            PC.dotHandles[i,1][:set_xdata](Vparams[end-(myX-1)])
            PC.dotHandles[i,1][:set_ydata](Vparams[end-(myY-1)])   
            PC.dotHandles[i,1][:set_visible](true)
        end
    end
    axes(PC.axisHandles[end])    
    legend(PC.dotHandles[end,1:3], ["current", "1 ago", "2 ago"])
    pause(0.001)
end


"""
    PCA_event_callback(xy, r, linehandle, axhandle, PC::PCAplot_data)

Internal function used by `PCA_plot()` to enable GUI interactivity.
This function is responsible for turning the position of the 
selected data point into the corresponding filename, and then
calling the callback that was registered with `PCA_plot()` (if any was)

Out of the entries in PC::PCAplot_data, this function uses axisHandles, axisPCs,
Vparams, files, and callback.

"""
function PCA_event_callback(xy, r, linehandle, axhandle, PC::PCAplot_data)
    # @printf("xy=(%g,%g)\n", xy[1], xy[2])
    idx = nothing
    # Let's go through the axes finding our axis
    for i=1:length(PC.axisHandles)
        if axhandle == PC.axisHandles[i]            
            myX = PC.axisPCs[i,1]; myY = PC.axisPCs[i,2]
            # and now find the index of the point at xy
            idx = find((PC.Vparams[:, end-(myX-1)].==xy[1]) .& (PC.Vparams[:, end-(myY-1)].==xy[2]))
            if length(idx)==0; 
                @printf("PCA_event_callback: Couldn't find point (%.3f,%.3f), returning\n", xy[1], xy[2]); 
                return; 
            end
            idx = idx[1]
        end
    end
    
    @printf("You selected file %s\n", PC.files[idx]); pause(0.0001)    
    
    # If there is a user callback, call it:
    if PC.callback != nothing
        PC.callback(PC.files[idx], PC)
    end        
end 
    


"""
    PC = plot_PCA(res; threshold=-0.0002, fignum=2, pc_offset=0, plot_unsuccessful=true,
        compute_good_only=true, unsuccessful_threshold=nothing, cost_choice="cost",
        user_callback=nothing)

Computes the principal components for a matrix of parameter values across many runs, and puts
up several scatterplots of the run parameter values projected onto these principal 
components. In addition, enables GUI interactivity: users can associate a callback function
with buttonclicks on the figure.

# PARAMETERS:

- res     A dictionary, in the format of the output of `farmload()`

# OPTIONAL PARAMETERS:

- threshold             costs below this are considered "successful" runs

- unsuccessful_threshold   costs above this are considered "UNsuccessful" runs
                        This value defaults to whatever threshold is.  If the two thresholds are
                        different, farms with costs in between, in the "no man's land", are not plotted.

- plot_unsuccessful     If true, dots for unsuccessful runs are shown, otherwise not.

- compute_good_only   If true, all runs, whether successful or not, are used to compute the PCs.
                        If false, only the successful runs.

- cost_choice           String, used to indicate which cost will be used for thresholding. It 
                        must be either "cost", indicating the testing cost, or "tcost", the training cost. 

- pc_offset             Shows PCs from pc_offset+1 to pc_offset+4

- user_callback         If set, function that will be called, as `usercallback(filename, PC)` where
                        filename is the name of the run that corresponds to the selected point.

- fignum                Figure number on which to plot the PC scatterplots

""" 
function plot_PCA(res; threshold=-0.0001, fignum=2, pc_offset=0, plot_unsuccessful=true,
    compute_good_only=true, unsuccessful_threshold=nothing, cost_choice="cost",
    user_callback=nothing)

    if unsuccessful_threshold==nothing
        unsuccessful_threshold = threshold
    end

    # Initialize a PCAplot_data structure
    PC = PCAplot_data([], [], [], [], [], [], [], [], [], [], [], [], nothing, nothing)

    # Get the runs' costs and do selection on them:
    if ~((cost_choice=="tcost")  ||  (cost_choice=="cost"))
        error("cost_choice MUST be one of \"tcost\" or \"cost\"")
    end    
    mycost = res[cost_choice]; 
    PC.I  = I  = find(mycost .< threshold)
    PC.nI = nI = find(mycost .>= unsuccessful_threshold)

    if compute_good_only
        # Use the successful runs (sparams) to define PCA space
        sparams = copy(res["params"][I,:])
        for i=1:size(sparams,2)
            sparams[:,i] -= mean(sparams[:,i])
            sparams[:,i] /= std(sparams[:,i])
        end
        C = sparams'*sparams/size(sparams,1)
        D,V = eig(C)
        pv = 100*D/sum(D)
    end

    # Then z-score everybody
    params = copy(res["params"]); nruns = size(params,1)
    PC.mu = mean(params,1);
    params = params - ones(nruns,1)*PC.mu
    PC.sd = std(params,1);
    params = params ./ (ones(nruns,1)*PC.sd)

    if !compute_good_only
        C = params'*params/nruns
        D,V = eig(C)
        pv = 100*D/sum(D)
    end
    PC.V = V

    # parameters in the eigen-coords:
    PC.Vparams = Vparams = (inv(V)*params')'

    figure(fignum); clf(); 
    # ax1 = subplot(2,2,1); axisHeightChange(0.9, lock="t")
    # ax2 = subplot(2,2,2); axisMove(0.05, 0); axisHeightChange(0.9, lock="t")
    # ax3 = subplot(2,2,3); axisHeightChange(0.9, lock="b")
    # ax4 = subplot(2,2,4); axisMove(0.05, 0); axisHeightChange(0.9, lock="b")
    # PC.axisHandles = [ax1, ax2, ax3, ax4]
    # PC.axisPCs = [1 2 ; 3 2 ; 1 3 ; 3 4] + pc_offset
    ax1 = subplot(1,2,1); ax2 = subplot(1,2,2)
    PC.axisHandles = [ax1 ; ax2]
    PC.axisPCs = [2 1 ; 3 1]

    for i=1:length(PC.axisHandles)
        axes(PC.axisHandles[i])
        myX = PC.axisPCs[i,1]; myY = PC.axisPCs[i,2]
        if plot_unsuccessful
            plot(Vparams[nI,end-(myX-1)],  Vparams[nI,end-(myY-1)], "g.", markersize=10)
        end
        plot(Vparams[I,end-(myX-1)],  Vparams[I,end-(myY-1)], "r.", markersize=10)
        title(@sprintf("PCA %d (%.2f%%) vs %d (%.2f%%)", myY, pv[end-(myY-1)], myX, pv[end-(myX-1)]))
    end

        
    # Add the non-red dots:
    hs =Array{PyCall.PyObject}(0, 3)
    for i=1:length(PC.axisHandles)
        axes(PC.axisHandles[i]); 
        hs = [hs ; reshape([plot(0, 0, "m.") plot(0, 0, "b.") plot(0, 0, "c.")][end:-1:1], 1, 3)]
        # order of handles gets reversed so the last one plotted -- goes on top -- is first handle
    end
    for h in hs; h[:set_markersize](11); h[:set_visible](false); end
    PC.dotHandles = hs
    legend(hs[end,1:3], ["current", "1 ago", "2 ago"])
    PC.callback = user_callback
    PC.files = res["files"]
        
    # axes(ax3)
    # if plot_unsuccessful; legend(["unsuccessful", "successful"])
    # else                   legend(["successful"]) 
    # end

    install_nearest_point_callback(figure(fignum), PCA_event_callback, user_data=PC)

    return PC 
end



"""
SV = plot_SVD(;threshold =-0.0002, plot_unsuccessful=false, compute_good_only=false,
        cost_choice="cost", user_callback=nothing, fignum=100)

Uses the pre-computed SVD components for a matrix of parameter values across many runs, and puts
up scatterplots of the run parameter values projected onto these SV components (columns of the U 
matrix. In addition, enables GUI interactivity: users can associate a callback function
with buttonclicks on the figure.


# OPTIONAL PARAMETERS:

- threshold             training costs below this value are considered "successful" (red dots), 
                        above it are "unsuccessful" (blue dots)

- plot_unsuccessful     If true, dots for unsuccessful runs are shown, otherwise not.

- compute_good_only     If true, only the successful runs are used to compute the SVD space

- cost_choice           String, used to indicate which cost will be used for thresholding. It 
                        must be either "cost", indicating the testing cost, or "tcost", the training cost. 

- user_callback         If set, function that will be called, as `usercallback(filename, SV)` where
                        filename is the name of the run that corresponds to the selected point.

- fignum                Figure number on which to plot the PC scatterplots


# RETURNS

- SV     A structure of type PCAplot_data. (Only its files, Vparams, axHandles, dotHandles
         axisPCs and callback entries will have assigned values.)

    
"""
function plot_SVD(;threshold =-0.0001, plot_unsuccessful=false, compute_good_only=false,
    cost_choice="cost", user_callback=nothing, fignum=100)

    # get response matrix
    response, results = load("SVD_response_matrix.jld", "response","results");
    # set up filter by nan
    nanrows = any(isnan.(response),2);

    # Get the runs' costs and do selection on them:
    if ~((cost_choice=="tcost")  ||  (cost_choice=="cost"))
        error("cost_choice MUST be one of \"tcost\" or \"cost\"")
    end    
    mycost = results[cost_choice]; 
    if !compute_good_only
        mycost = mycost[.!vec(nanrows),:];
        disp_cost = copy(mycost);
    end
    badcost = mycost .>= threshold;
    
    # if we are computing SVD only on the good farms, update nanrows and disp_cost
    if compute_good_only
        nanrows = nanrows .| badcost;
        disp_cost = mycost[.!vec(nanrows),:];
    end    

    # Filter response matrix
    r_all = response[.!vec(nanrows),:];
    m = mean(r_all,1);
    r_all = r_all - repmat(m, size(r_all,1),1);
    F = svdfact(r_all);
    u = copy(F[:U]); 
    u1 = u[:,1];
    u2 = u[:,2];
    u3 = u[:,3];

    # Make list of just good farms
    if compute_good_only
        # nanrows filter already selects for good farms
        u1good = u1;
        u2good = u2;
        u3good = u3;
    else
        # remove bad cost farms
        u1good = u1[.!vec(badcost),:];
        u2good = u2[.!vec(badcost),:];
        u3good = u3[.!vec(badcost),:];
    end
    files = results["files"];
    files = files[.!vec(nanrows),:];

    SV = PCAplot_data([], [], [], [], [], [], [], [], [], [], [], [], nothing, nothing)
    SV.files = files
    SV.Vparams = u[:,3:-1:1]  # The column vectors are expected in REVERSE order (i.e., as returned by eig(). )
    
    pygui(true)
    figure(fignum); clf();

    ax1 = subplot(1,2,1)
    if plot_unsuccessful; plot(u[:,3],u[:,1],"bo"); end
    plot(u3good, u1good, "ro")
    title("SVD U columns 3 and 1")
    ylabel("SVD Dim 1")
    xlabel("SVD Dim 3")   
    plot(0, 0, "go")[1][:set_visible](false)

    ax2 = subplot(1,2,2)
    if plot_unsuccessful; plot(u[:,2],u[:,1],"bo"); end
    plot(u2good, u1good, "ro")
    title("SVD U columns 2 and 1")
    xlabel("SVD Dim 2")   
    remove_ytick_labels()

    SV.axisHandles = [ax1 ; ax2]
    SV.axisPCs   = [3 1 ; 2 1]

    hs =Array{PyCall.PyObject}(0, 3)
    for i=1:length(SV.axisHandles)
        axes(SV.axisHandles[i]); 
        hs = [hs ; reshape([plot(0, 0, "m.") plot(0, 0, "b.") plot(0, 0, "c.")][end:-1:1], 1, 3)]
        # order of handles gets reversed so the last one plotted -- goes on top -- is first handle
    end
    for h in hs; h[:set_markersize](11); h[:set_visible](false); end
    SV.dotHandles = hs
    legend(hs[end,1:3], ["current", "1 ago", "2 ago"])

    SV.callback = user_callback
    # [Out of the entries in PC::PCAplot_data, this function uses files,  Vparams, axHandles, dothandles, and axisPCs]

    install_nearest_point_callback(figure(fignum), PCA_event_callback, user_data=SV)

    return SV
end




plot_farm_trials = 10    #  The number of trials to be plotted per farm run

"""
    params = plot_farm(filename; testruns=nothing, fignum=3, overrideDict=Dict())

    Plots multiple trials from a single run of a farm.

# PARAMETERS

- filename    The filename of the .jld file containing the run, to be loaded

# OPTIONAL PARAMETERS

- testruns    Number of trials to run. Defaults to value of global variable plot_farm_trials.

- fignum      Figure to put the plot up in.

- overrideDict   A dictionary containing any model parameter values that will
              override any values loaded from the file.  For example
              `overrideDict = Dict(:sigma=>0.001)` will run with that value
              of sigma, no whater what the file said.

"""
function plot_farm(filename; testruns=400, fignum=3, overrideDict=Dict())

    mypars, extra_pars, args, pars3 = load(filename, "mypars", "extra_pars", "args", "pars3")


    pygui(true)
    figure(fignum); clf();
    
    pstrings = ["CONTROL", "DELAY OPTO", "CHOICE OPTO"]
    for period = 1:3
        these_pars = merge(mypars, extra_pars);
        these_pars = merge(these_pars, Dict(
        # :opto_strength=>0.3, 
        :opto_times=>reshape(extra_pars[:opto_periods][period,:], 1, 2),
        # :opto_times=>["target_start-0.4" "target_start"],
        # :opto_times=>["target_start" "target_end"],
        # :post_target_period=>0.3,
        # :rule_and_delay_period=>1.2,
        # :dt=>0.005,
        ))

        # The plot_list should be the one we give it below, not whatever was in the stored parameters
        delete!(these_pars, :plot_list)

        pvax = subplot(4,3,period);   axisHeightChange(0.9, lock="t")
        pdax = subplot(4,3,period+3); axisHeightChange(0.9, lock="c"); 
        avax = subplot(4,3,period+6); axisHeightChange(0.9, lock="c")
        adax = subplot(4,3,period+9); axisHeightChange(0.9, lock="b")

        proVs, antiVs = run_ntrials(testruns, testruns; plot_list=[1:20;], plot_Us=false, 
            ax_set = Dict("pro_Vax"=>pvax, "pro_Dax"=>pdax, "anti_Vax"=>avax, "anti_Dax"=>adax),
        merge(make_dict(args, pars3, these_pars), overrideDict)...);

        hBP = length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
        hBA = length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)
        # @printf("period %d:  hBP=%.2f%%, hBA=%.2f%%\n\n", period, 100*hBP, 100*hBA)

        axes(pvax); title(@sprintf("%s  PRO hits = %.2f%%", pstrings[period], 100*hBP))
        axes(avax); title(@sprintf("ANTI hits = %.2f%%", 100*hBA))
        axes(pdax); remove_xtick_labels(); xlabel("")
        if period > 1
            remove_ytick_labels([pvax, pdax, avax, adax])
        end
        
        figure(fignum)[:canvas][:draw]()
        pause(0.0001)
    end

    for a=1:length(args)
        myarg = args[a]; while length(myarg)<20; myarg=myarg*" "; end
        @printf("%s\t\t%g\n", myarg, pars3[a])
    end

    return pars3
end


plot_farm

In [90]:

#######################################################
#
#     Actual calls to the functions to run everything
#
#######################################################



if !isdefined(:res)
    res = farmload("C17", verbose=true, farmdir="MiniFarms")
end

pygui(true); 
remove_all_BPs(); # Clean up any previous links between clicks on figures and callback functions
plt[:close](1); plt[:close](2); plt[:close](3); plt[:close](4); plt[:close](5)

# Carlos' favored configuration, but adjust to suit -- use capture_current_figure_configuration() 
# to see code that reproduces a configuration you like once you find it

figure(1); set_current_fig_position(1325, 41, 640, 982)   # x, y, width, height
figure(2); set_current_fig_position(645, 785, 680, 408)
figure(3); set_current_fig_position(0, 785, 641, 407)
figure(4); set_current_fig_position(3, 23, 1288, 797)
figure(5); set_current_fig_position(1338, 998, 540, 200)

figure(5)
ax1 = subplot(2,2,1); axisMove(-0.05, 0); axisWidthChange(1.1, lock="l")
ax2 = subplot(2,2,2); axisMove(0.05, 0); axisWidthChange(0.6, lock="r")
ax3 = subplot(2,1,2); axisMove(0.05, 0)
rad = kbMonitorModule.radio_buttons(ax1, ["Don't plot trials", "Plot trials (wait for it)"])
tbx = kbMonitorModule.text_box(ax2, "ntrials to run ", "10")
dbx = kbMonitorModule.text_box(ax3, "Override ", "")


HD = histo_params(res; threshold=-0.0001, cost_choice="cost");
pause(0.001)

# The callback function that will be called after clicking on a data dot:
function highlight_all(fname, PC, SV)
    PCA_highlight(fname, PC);   # Color the selected dots in the PC plot
    PCA_highlight(fname, SV);   # Color the selected dots in the SVD plot
    histo_highlight(fname, HD)  # Color the selected bars in the histograms
    pause(0.001);   # We don't really care about the 1 ms pause; just a convenient way to flush all pending graphics commandsj
    if rad[:value_selected] == "Plot trials (wait for it)"
        if ~isnull(tryparse(Int64, tbx[:text])); 
            ntrials = parse(Int64, tbx[:text])
        else
            @printf("Couldn't parse the ntrials to plot\n")
            ntrials = 10
        end
        try
            plot_farm(fname, testruns=ntrials, fignum=4, overrideDict=eval(parse("Dict("*dbx[:text]*")"))) 
        catch e
            @printf("Couldn't plot farm, error %s", e)
        end
    end
end

PC = plot_PCA(res; threshold=-0.0001, cost_choice="cost", 
    user_callback = (fname, Trash) -> highlight_all(fname, PC, SV),
    plot_unsuccessful=false, unsuccessful_threshold=0.0001, compute_good_only=true, fignum=2);
pause(0.001)

SV = plot_SVD(threshold=-0.0001, cost_choice="cost", 
    user_callback = (fname, Trash) -> highlight_all(fname, PC, SV),
    plot_unsuccessful=false, compute_good_only=true, fignum=3);

docstring = """

Wait for PCA and SVD plots to come up.
Then click on any dot within the PCA plot or SVD plots to
see the corresponding data in the other plots. Click on
'Don't plot trials' in figure 5 if you want to go faster,
without running trials.

The window placement fits a 15-in Macbook Pro, but adjust
at will. Once you find window positions you like, run
    capture_current_figure_configuration()
to get copy-pastable code that reproduces it.

ntrials to run is the number of trials per condition that will
be run in plot_farm() to compute %correct. Up to 20 of them
will be plotted.

Override is a dict that will override any paramater values in
the plotted trials. For example, if you write
    :sigma=>0.2, :rule_and_delay_period=>1.6
then no matter what the run's file said, if you ask for trials
to be plotted, those trials will run
with that sigma and that rule_and_delay_period.

"""

@printf("%s", docstring)



Wait for PCA and SVD plots to come up.
Then click on any dot within the PCA plot or SVD plots to
see the corresponding data in the other plots. Click on
'Don't plot trials' in figure 5 if you want to go faster,
without running trials.

The window placement fits a 15-in Macbook Pro, but adjust
at will. Once you find window positions you like, run
    capture_current_figure_configuration()
to get copy-pastable code that reproduces it.

ntrials to run is the number of trials per condition that will
be run in plot_farm() to compute %correct. Up to 20 of them
will be plotted.

Override is a dict that will override any paramater values in
the plotted trials. For example, if you write
    :sigma=>0.2, :rule_and_delay_period=>1.6
then no matter what the run's file said, if you ask for trials
to be plotted, those trials will run
with that sigma and that rule_and_delay_period.



QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap


You selected file MiniFarms/farm_C17_Farms024_0646.jld
anti_rule_strength  		0.0497992
right_light_excitation		1.22284
sW                  		1.68559
sigma               		-0.167337
opto_strength       		0.700664
const_pro_bias      		0.0239837
dW                  		0.325379
target_period_excitation		-0.674757
hW                  		0.23944
vW                  		-0.623042
pro_rule_strength   		0.0156859
constant_excitation 		-0.703305
You selected file MiniFarms/farm_C17_Farms024_0434.jld
anti_rule_strength  		0.0682838
right_light_excitation		1.54045
sW                  		1.48858
sigma               		0.216982
opto_strength       		0.496767
const_pro_bias      		0.0308313
dW                  		0.407145
target_period_excitation		-0.953245
hW                  		0.290716
vW                  		-0.578602
pro_rule_strength   		0.0192155
constant_excitation 		-0.688311
You selected file MiniFarms/farm_C17_Farms026_0048.jld
anti_rule_strength  		0.0864679
right_light_excitation		0.379255
sW    

In [85]:
testruns=800
mypars, extra_pars, args, pars3 = load("MiniFarms/farm_C17_Farms024_0430.jld", "mypars", "extra_pars", "args", "pars3")

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(testruns, testruns; verbose=false, 
    make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)

Opto condition # 1
     - 1 - cost=-0.00014407, cost1=6.12756e-05, cost2=-0.000205346
     - 1 - mean(hitsP)=0.896437, mean(diffsP)=0.988445 mean(hitsA)=0.71884, mean(diffsA)=0.243629
Opto condition # 2
     - 2 - cost=-0.000130244, cost1=7.14198e-06, cost2=-0.000137386
     - 2 - mean(hitsP)=0.845757, mean(diffsP)=0.799782 mean(hitsA)=0.495015, mean(diffsA)=0.0245355
Opto condition # 3
     - 3 - cost=-0.000196253, cost1=4.38879e-06, cost2=-0.000200642
     - 3 - mean(hitsP)=0.896813, mean(diffsP)=0.98876 mean(hitsA)=0.695978, mean(diffsA)=0.215094
OVERALL
     -- cost=-0.000156856, cost1=2.42688e-05, cost2=-0.000181125


(-0.0001568559338562538, [0.000183827 0.0 0.0; 2.14259e-5 0.0 0.0; 1.31664e-5 0.0 0.0], [-0.000616037 0.0 0.0; -0.000412159 0.0 0.0; -0.000601927 0.0 0.0], [0.896437; 0.845757; 0.896813], [0.71884; 0.495015; 0.695978], [0.988445; 0.799782; 0.98876], [0.243629; 0.0245355; 0.215094], [0.89875; 0.87; 0.895], [0.58; 0.09875; 0.53875])

In [88]:
[hBP hBA]

3×2 Array{Float64,2}:
 0.89875  0.58   
 0.87     0.09875
 0.895    0.53875

You selected file MiniFarms/farm_C17_Farms025_0288.jld
anti_rule_strength  		0.130757
right_light_excitation		0.112683
sW                  		2.16737
sigma               		0.0058295
opto_strength       		0.904628
const_pro_bias      		-0.0370506
dW                  		0.48759
target_period_excitation		-0.709303
hW                  		-0.0822871
vW                  		-0.880882
pro_rule_strength   		0.176699
constant_excitation 		-0.790764
overrideDict was Dict()
You selected file MiniFarms/farm_C17_Farms024_0453.jld
anti_rule_strength  		0.14902
right_light_excitation		0.517539
sW                  		1.90388
sigma               		-0.00219335
opto_strength       		0.893178
const_pro_bias      		0.0439513
dW                  		0.251438
target_period_excitation		-0.445429
hW                  		-0.232844
vW                  		-0.218031
pro_rule_strength   		0.109863
constant_excitation 		-0.757036
overrideDict was Dict()
You selected file MiniFarms/farm_C17_Farms025_0081.jld
anti_rule_strength 

In [86]:
ppps = plot_farm("MiniFarms/farm_C17_Farms024_0430.jld", testruns=800, fignum=10)

anti_rule_strength  		0.118424
right_light_excitation		0.441949
sW                  		2.02788
sigma               		0.00408345
opto_strength       		0.979051
const_pro_bias      		-6.46556e-05
dW                  		0.201323
target_period_excitation		-0.5608
hW                  		-0.121376
vW                  		-0.35398
pro_rule_strength   		0.121726
constant_excitation 		-0.761411


QPixmap::scaled: Pixmap is a null pixmap


12-element Array{Float64,1}:
  0.118424  
  0.441949  
  2.02788   
  0.00408345
  0.979051  
 -6.46556e-5
  0.201323  
 -0.5608    
 -0.121376  
 -0.35398   
  0.121726  
 -0.761411  

In [46]:
figure(5); clf();
ax1 = subplot(2,2,1); axisMove(-0.05, 0); axisWidthChange(1.1, lock="l")
ax2 = subplot(2,2,2); axisMove(0.05, 0); axisWidthChange(0.6, lock="r")
ax3 = subplot(2,1,2); axisMove(0.05, 0)
rad = kbMonitorModule.radio_buttons(ax1, ["Don't plot trials", "Plot trials (wait for it)"])
tbx = kbMonitorModule.text_box(ax2, "ntrials to run ", "10")
dbx = kbMonitorModule.text_box(ax3, "Override ", "")


PyObject <matplotlib.widgets.TextBox object at 0x1609152b0>

In [35]:
axes(ax1)
ylim(0, 2)

(0, 2)

In [ ]:
res = farmload("C17", verbose=true, farmdir="MiniFarms")

res["args"]

In [ ]:
HD = histo_params(res, threshold=0.0001, fignum=1)
remove_all_BPs(); SV = plot_SVD(user_callback=
    (fname, PC) -> begin PCA_highlight(fname, PC); histo_highlight(fname, HD); end);

In [ ]:

load("MiniFarms/farm_C17_Farms024_0001.jld", "args")

In [ ]:
include("general_utils.jl")

In [ ]:
BP = __permanent_BP_store[1]
BP[:buttonlist]()

In [ ]:

"""
    make_mini_farm()

Takes the C17 runs in ../Farms024, ../Farms025, and ../Farms026, which are not on git,
and puts a small-size sumamry of them in "MiniFarms/". That MiniFarms directory is
a reasonabale size for git (only 63MBytes compred to GBytes)

The MiniFarms directory can be used for the C17_browser in lieu of the original farms.

"""
function make_min_farm()
    
    res = farmload("C17", verbose=true, farmdir=["../Farms024", "../Farms025", "../Farms026"])

    if ~isdir("MiniFarms"); mkdir("MiniFarms"); end;

    sdict = []; dirname=[]; filename=[]

    for i in 1:length(res["tcost"])    
        mypars, extra_pars, args, pars3 = load(res["files"][i], "mypars", "extra_pars", 
            "args", "pars3")
        sdict = Dict("mypars"=>mypars, "extra_pars"=>extra_pars, 
            "args"=>args, "pars3"=>pars3)
        for k in keys(res)
            if k=="tcost"
                sdict["traj3"] = [0 ; res["tcost"][i]; 0]
            elseif !(k=="args" || k=="params")
                sdict[k] = res[k][i]
            end
        end

        dirname, filename = splitdir(res["files"][i]); dirname=splitdir(dirname)[2]

        save("MiniFarms/"*filename[1:9]*dirname*"_"*filename[10:end], sdict)
        if rem(i, 20)==0
            @printf("Did %d/%d\n", i, length(res["tcost"]))
        end
    end
end



In [ ]:
remove_all_BPs()

In [ ]:
histo_params(res; threshold=-0.0001, nbins=10, fignum=1)

In [ ]:

####################################################################
#                                                                  
#   Now use farmload() to load a summary of the farms into variable res
#   run histo_params() on it to put histograms up on figure 1
#
#   Note that default for histo_params() and plot_PCA() is to use training
#   cost ("tcost") not testing cost ("cost)
#
####################################################################

# --- Load the data, plot histograms, and show the PCA scatter

# The following line 
# res = farmload("C17", verbose=true, farmdir=["../Farms024" "../Farms025" "../Farms026"])
res = farmload("C17", verbose=true, farmdir="MiniFarms")

threshold = -0.0002  # Cost below this is needed to count as a successful run 

histo_params(res; threshold=threshold, nbins=10, fignum=1)
I  = find(res["tcost"].<threshold)
nI = find(res["tcost"] .>= threshold)

@printf("There were %d/%d (%.2f%%) successful runs\n", length(I), length(res["tcost"]), 
100*length(I)/length(res["tcost"]))



In [ ]:
figure(5); clf()
I = find(res["cost"].<0.0001)
subplot(1,2,1); plot(res["tcost"][I,:], res["cost"][I,:], "b.")
subplot(1,2,2); plt[:hist](res["cost"][I,:]-res["tcost"][I,:]);

cla();
plt[:hist](res["cost"][I,:]', 40);
size(I)

In [ ]:

####################################################################
#                                                                  
#   MAIN DATA BROWSER FUNCTION
#
####################################################################


global BP = []    # Declare this outside so the BP variable is available outside the function

"""
    restart_figure2(res ; testruns=20)

Given a variable res containing a summary of farm runs (as produced by farmload()), 
puts up a plot of run paramaters in the first four PCs, and makes it clickable:
clicking on any of the points in the PCA space will brung up testrun individual new runs
using plot_farm() in figure 3.

"""
function restart_figure2(res ; testruns=20, use_all_runs_for_PCA=false)
    # First put up the PCA plot
    C, V, D, Vparams, I, nI  = plot_PCA(res; threshold=threshold,
        use_all_runs_for_PCA=use_all_runs_for_PCA,
        pc_offset=0, plot_unsuccessful=false, fignum=2);

    files = res["files"]
    mu = mean(res["params"],1)
    sd = std( res["params"],1)


    # Set up initial blue, green, and magenta dots; as the user clicks
    # the will follow along
    hs = []; 
    figure(2)
    ax1 = subplot(2,2,1); hs = [hs ; [plot(0, 0, "b.") plot(0, 0, "g.") plot(0, 0, "m.")]]
    ax2 = subplot(2,2,2); hs = [hs ; [plot(0, 0, "b.") plot(0, 0, "g.") plot(0, 0, "m.")]]
    ax3 = subplot(2,2,3); hs = [hs ; [plot(0, 0, "b.") plot(0, 0, "g.") plot(0, 0, "m.")]]
    ax4 = subplot(2,2,4); hs = [hs ; [plot(0, 0, "b.") plot(0, 0, "g.") plot(0, 0, "m.")]]


    # Now the function that will get called after every key press or button press in the PCA
    # figure
    function event_callback()
        # Get the list of button presses in the figure:
        bpe = BP[:buttonlist]()
        # Remove any leading buttonpresses that were not within axes:
        while length(bpe)>0 && bpe[1][1] == nothing
            bpe = bpe[2:end]
        end
        # If there are any remaining button presses, deal with them:
        if length(bpe)>0        

            # We know which axes go with which principal components from knowing how
            # plot_PCA is written-- but it would be better if we didn't have to know
            # that in order to write this function:
            #  J will be a list of squared distances to points on the clicked axis
            if bpe[1][1]==ax1
                J = (Vparams[I,end]   - bpe[1][2]).^2 + (Vparams[I,end-2] - bpe[1][3]).^2            
            elseif bpe[1][1]==ax2
                J = (Vparams[I,end-1] - bpe[1][2]).^2 + (Vparams[I,end-3] - bpe[1][3]).^2
            elseif bpe[1][1]==ax3
                J = (Vparams[I,end]   - bpe[1][2]).^2 + (Vparams[I,end-1] - bpe[1][3]).^2
            elseif bpe[1][1]==ax4
                J = (Vparams[I,end-2] - bpe[1][2]).^2 + (Vparams[I,end-3] - bpe[1][3]).^2
            else
                error("Which axes did this buttonpress come from???")
            end
            # We're going to go to the red point closest to the clicked position
            idx = indmin(J)

            @printf("---- FILE %s: -----\n", files[I[idx]])
            pars = plot_farm(files[I[idx]], testruns=testruns, fignum=3)
            # pars = load(files[I[idx]], "pars3")
            pars = (pars-mu')./sd'
            vpars = inv(V)*pars
            # print("\n"); print(vpars); print("\n")

            # The dot in column 3 will get the coords of the dot in column 2;
            # the dot on column 2 will get the coords of the dot in column 1.
            for row=1:4
                for from=2:-1:1
                    to = from+1
                    hs[row,to][:set_xdata](hs[row,from][:get_xdata]())
                    hs[row,to][:set_ydata](hs[row,from][:get_ydata]())
                end
            end

            # And then the dot in column 1 gets the coords of the red dot closestto the clicked point:
            hs[1,1][:set_xdata](vpars[end  ]); hs[1,1][:set_ydata](vpars[end-2])
            hs[2,1][:set_xdata](vpars[end-1]); hs[2,1][:set_ydata](vpars[end-3])
            hs[3,1][:set_xdata](vpars[end  ]); hs[3,1][:set_ydata](vpars[end-1])
            hs[4,1][:set_xdata](vpars[end-2]); hs[4,1][:set_ydata](vpars[end-3])

            # If we just sorted out a button press on figure 2, (e.g., ran plot_farm()
            # onto figure 3), now make sure figure 2 
            # is the current figure again after having dealt with the button press:
            figure(2)
        end
        # Clear the button press list after we have dealt with it:
        BP[:clear_buttonlist]()
    end

    # When we are first run, set up the button press monitor:
    global BP = kbMonitorModule.kb_monitor(figure(2), callback=event_callback)
    legend(hs[4,1:3], ["current", "1 ago", "2 ago"])
end

# Bring up PCA plot in figure 2 and make it clickable:
restart_figure2(res)


@printf("\n\nClicking on any red dot in figure 2 will bring up 20 trials of the\n")
@printf("corresponding run on figure 3.\n\n")
@printf("The blue dot is the current farm displayed; green is one click ago;\n")
@printf("and magenta is two clicks ago.\n\n")

In [ ]:
restart_figure2(res, use_all_runs_for_PCA=true)

In [ ]:
pars = plot_farm("MiniFarms/farm_C17_Farms024_0001.jld", testruns=20, fignum=3)

In [ ]:
I = find(res["tcost"].<-0.0002)
files = res["files"][I,:]

mu = mean(res["params"],1)
sd = std( res["params"],1)

apars = copy(res["params"][I,:])
apars = apars - ones(size(apars,1),1)*mu
apars = apars ./ (ones(size(apars,1))*sd)
avpars = (inv(V)*apars')'

P = sortperm(avpars[:,end])
1

In [ ]:
I = find(res["tcost"].<-0.0002)
files = res["files"][I,:]

mu = mean(res["params"],1)
sd = std( res["params"],1)

apars = copy(res["params"][I,:])
apars = apars - ones(size(apars,1),1)*mu
apars = apars ./ (ones(size(apars,1))*sd)
avpars = (inv(V)*apars')'

P = sortperm(avpars[:,end])

for k=1:5:length(P)

    @printf("---- FILE %s: -----\n", files[P[k]])
    pars = plot_farm(files[P[k]], testruns=20, fignum=3)
    pars = (pars-mu')./sd'
    vpars = inv(V)*pars
    
    hs = []
    figure(2)
    subplot(2,2,1); hs = [hs ; plot(vpars[end], vpars[end-2], "b.")]
    subplot(2,2,3); hs = [hs ; plot(vpars[end], vpars[end-1], "b.")]
    subplot(2,2,2); hs = [hs ; plot(vpars[end-1], vpars[end-3], "b.")]
    subplot(2,2,4); hs = [hs ; plot(vpars[end-2], vpars[end-3], "b.")]
    
    ans = chomp(readline())
    if startswith(ans, "q") || startswith(ans, "Q")
        for ii in hs; ii[:remove](); end
        break
    end

    for ii in hs; ii[:remove](); end        
end


# h = plot(0, -0.1, "b.")

# Ls = gca()[:lines]
# h[1][:remove]()

In [ ]:
PyCall.pyversion

In [ ]:
BP = myModule.kb_monitor(figure(2))

In [ ]:
hs = plot(0, 0, "b.")

In [ ]:
BP[:clear_buttonlist]()

In [ ]:
for i in hs; i[:set_color]("y"); end
figure(2)[:canvas][:draw]()

In [ ]:
PyPlot.show()

In [ ]:
files = selectize(res, find(res["tcost"].<-0.0002))["files"]

for f in files
    plot_farm(f, testruns=200)
    
    ans = chomp(readline())
    if startswith(ans, "q") || startswith(ans, "Q")
        break
    end
end


In [ ]:
f = "farm_C17_0032.jld"
root = f[1:end-8]

## Look at individual trials

In [ ]:
args = res["args"]
I= find((res["tcost"].<-0.0002))
res["files"][I]

In [83]:
testruns=800
mypars, extra_pars, args, pars3 = load("MiniFarms/farm_C17_Farms024_0430.jld", "mypars", "extra_pars", "args", "pars3")

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(testruns, testruns; verbose=false, 
    make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)

Opto condition # 1
     - 1 - cost=-0.00014407, cost1=6.12756e-05, cost2=-0.000205346
     - 1 - mean(hitsP)=0.896437, mean(diffsP)=0.988445 mean(hitsA)=0.71884, mean(diffsA)=0.243629
Opto condition # 2
     - 2 - cost=-0.000130244, cost1=7.14198e-06, cost2=-0.000137386
     - 2 - mean(hitsP)=0.845757, mean(diffsP)=0.799782 mean(hitsA)=0.495015, mean(diffsA)=0.0245355
Opto condition # 3
     - 3 - cost=-0.000196253, cost1=4.38879e-06, cost2=-0.000200642
     - 3 - mean(hitsP)=0.896813, mean(diffsP)=0.98876 mean(hitsA)=0.695978, mean(diffsA)=0.215094
OVERALL
     -- cost=-0.000156856, cost1=2.42688e-05, cost2=-0.000181125


(-0.0001568559338562538, [0.000183827 0.0 0.0; 2.14259e-5 0.0 0.0; 1.31664e-5 0.0 0.0], [-0.000616037 0.0 0.0; -0.000412159 0.0 0.0; -0.000601927 0.0 0.0], [0.896437; 0.845757; 0.896813], [0.71884; 0.495015; 0.695978], [0.988445; 0.799782; 0.98876], [0.243629; 0.0245355; 0.215094], [0.89875; 0.87; 0.895], [0.58; 0.09875; 0.53875])

In [ ]:
pygui(true)
figure(1); clf();
figure(2); clf();

these_pars = merge(mypars, extra_pars);
these_pars = merge(these_pars, Dict(
# :opto_strength=>0.3, 
:opto_times=>reshape(extra_pars[:opto_periods][2,:], 1, 2),
# :opto_times=>["target_start-0.4" "target_start"],
# :opto_times=>["target_start" "target_end"],
:post_target_period=>0.3,
:rule_and_delay_period=>1.2,
:dt=>0.005,
))
delete!(these_pars, :plot_list)

override = Dict(
# :sigma=>0.02,
)

proVs, antiVs = run_ntrials(20, 20; plot_list=[1:20;], plot_Us=false, 
    merge(make_dict(args, pars3, these_pars), override)...);
hBP = length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
hBA = length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)
@printf("hBP=%.2f%%, hBA=%.2f%%\n", 100*hBP, 100*hBA)

figure(1); subplot(2,1,1)
title(@sprintf("PRO hits = %.2f%%", 100*hBP))
figure(2); subplot(2,1,1)
title(@sprintf("ANTI hits = %.2f%%", 100*hBA))

[args pars3]

In [ ]:
pygui(true)
figure(1); clf();
figure(2); clf();

these_pars = merge(mypars, extra_pars);
these_pars = merge(these_pars, Dict(
# :opto_strength=>0.3, 
:opto_times=>reshape(extra_pars[:opto_periods][2,:], 1, 2),
# :opto_times=>["target_start-0.4" "target_start"],
# :opto_times=>["target_start" "target_end"],
:post_target_period=>0.3,
:rule_and_delay_period=>1.2,
:dt=>0.005,
))
delete!(these_pars, :plot_list)

override = Dict(
# :sigma=>0.02,
)

proVs, antiVs = run_ntrials(100, 100; plot_list=[1:20;], plot_Us=false, 
    merge(make_dict(args, pars3, these_pars), override)...);
hBP = length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
hBA = length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)
@printf("hBP=%.2f%%, hBA=%.2f%%\n", 100*hBP, 100*hBA)

figure(1); subplot(2,1,1)
title(@sprintf("PRO hits = %.2f%%", 100*hBP))
figure(2); subplot(2,1,1)
title(@sprintf("ANTI hits = %.2f%%", 100*hBA))


In [ ]:
clf();
ax = gca();
axisHeightChange(0.5, ax=ax, lock="b")

In [ ]:
for k in keys(ax)
    print(k); print("\n")
end


In [ ]:
ax[:set_position]([0.1, 0.1, 0.1, 0.1])
b = ax[:get_position]()[:bounds]


In [ ]:
figure(2); clf()

using StatsBase

x = -0.001:0.000001:maximum(tcost)

h1 = fit(Histogram, tcost[:,1], x, closed=:right)

plot(x, [0 ; cumsum(h1.weights)]/sum(h1.weights), "b-")

xlabel("Training cost")
ylabel("fraction of runs")


In [ ]:
find(tcost.<-0.00015)

In [ ]:
# farm_id = "C11"; farmdir = "../NewFarms"

"""
    results, files, tcosts, qu_outs, sigmas, target_excites = lookem(;farm_id="C11", farmdir="../NewFarms")
"""
function lookem(;farm_id="C11", farmdir="../NewFarms")
    results = zeros(0,2); files =[]; qu_outs=[]; sigmas=[]; target_excites=[]; tcosts=[];
    for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(farmdir * "/"))
        d = load(farmdir * "/" * f)
        if !haskey(d, "qu_out"); d["qu_out"] = [0 0]; end;
        traj3 = d["traj3"]; qu_out = d["qu_out"]
        @printf("%s : sigma=%.3f, q=%.1f, %g\n", f, d["pars3"][7], qu_out[1], traj3[2,end])
        results = [results ; qu_out[1] traj3[2,end]]
        files = [files ; f]
        qu_outs = [qu_outs ; d["qu_out"][1]]
        sigmas  = [sigmas ; d["pars3"][7]]
        target_excites = [target_excites ; d["pars3"][2]]
        tcosts  = [tcosts ; traj3[2,end]]
    end
    return results, files, tcosts, qu_outs, sigmas, target_excites
end

res14 = lookem(farm_id="C14")

In [ ]:
[args pars3]

In [ ]:
find(res[:,2].<-0.0002)

In [ ]:
res[res[:,2].<-0.0002,:]

# Looking at individual trials of individual farms

In [ ]:
res16 = farmload("C16")
res16["params"][:,5]

threshold = -0.0002;
I = find(res16["tcost"].<threshold)
N = length(res16["tcost"])
@printf("You get %d/%d (%.2f%%) runs at training cost < %g\n", length(I), N, 100*length(I)/N, threshold)

In [ ]:
res16["args"]

In [ ]:
I= find((res16["tcost"].<-0.0002) .& (res16["params"][:,5].<1) .& (res16["params"][:,1].>0) .& (res16["params"][:,11].>0))
res16["files"][I]

In [ ]:
testruns=10
mypars, extra_pars, args, pars3 = load("../NewFarms/farm_C16_0175.jld", "mypars", "extra_pars", "args", "pars3")

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(testruns, testruns; verbose=false, 
    make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)

pygui(true)
figure(1); clf();
figure(2); clf();

these_pars = merge(mypars, extra_pars);
these_pars = merge(these_pars, Dict(
# :opto_strength=>0.3, 
:opto_times=>reshape(extra_pars[:opto_periods][1,:], 1, 2),
# :opto_times=>["target_start-0.4" "target_start"],
# :opto_times=>["target_start" "target_end"],
:post_target_period=>0.3,
:rule_and_delay_period=>1.2,
:dt=>0.005,
))
delete!(these_pars, :plot_list)

override = Dict(
# :sigma=>0.02,
)

proVs, antiVs = run_ntrials(100, 100; plot_list=[1:20;], plot_Us=false, 
    merge(make_dict(args, pars3, these_pars), override)...);
hBP = length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
hBA = length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)
@printf("hBP=%.2f%%, hBA=%.2f%%\n", 100*hBP, 100*hBA)

figure(1); subplot(2,1,1)
title(@sprintf("PRO hits = %.2f%%", 100*hBP))
figure(2); subplot(2,1,1)
title(@sprintf("ANTI hits = %.2f%%", 100*hBA))

[args pars3]

In [ ]:
pygui(true)
figure(1); clf();
figure(2); clf();

these_pars = merge(mypars, extra_pars);
these_pars = merge(these_pars, Dict(
# :opto_strength=>0.3, 
:opto_times=>reshape(extra_pars[:opto_periods][2,:], 1, 2),
# :opto_times=>["target_start-0.4" "target_start"],
# :opto_times=>["target_start" "target_end"],
:post_target_period=>0.3,
:rule_and_delay_period=>1.2,
:dt=>0.005,
))
delete!(these_pars, :plot_list)

override = Dict(
# :sigma=>0.02,
)

proVs, antiVs = run_ntrials(100, 100; plot_list=[1:20;], plot_Us=false, 
    merge(make_dict(args, pars3, these_pars), override)...);
hBP = length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
hBA = length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)
@printf("hBP=%.2f%%, hBA=%.2f%%\n", 100*hBP, 100*hBA)

figure(1); subplot(2,1,1)
title(@sprintf("PRO hits = %.2f%%", 100*hBP))
figure(2); subplot(2,1,1)
title(@sprintf("ANTI hits = %.2f%%", 100*hBA))


In [ ]:
mypars[:dt]

In [ ]:
res=lookem(farm_id="C11")

In [ ]:
load("../NewFarms/farm_C11_0001.jld", "README")

In [ ]:
# farm_id = "C11"; farmdir = "../NewFarms"

function lookem(;farm_id="C11", farmdir="../NewFarms")
    results = zeros(0,2)
    for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(farmdir * "/"))
        d = load(farmdir * "/" * f)
        if !haskey(d, "qu_out"); d["qu_out"] = [0 0]; end;
        traj3 = d["traj3"]; qu_out = d["qu_out"]
        @printf("%s : sigma=%.3f, q=%.1f, %g\n", f, d["pars3"][7], qu_out[1], traj3[2,end])
        results = [results ; qu_out[1] traj3[2,end]]
    end
    return results
end

res = lookem(farm_id="C12")

In [ ]:
find(res[:,2].<-0.0001)

In [ ]:
load("../NewFarms/farm_C12_0144.jld", "bbox")

In [ ]:
function change_farm_name(from, to; farmdir="../NewFarms")
    for f in filter(x -> startswith(x, "farm_" * from * "_"), readdir(farmdir))
        myfrom = farmdir * "/" * f
        myto   = farmdir * "/" * replace(f, "_"*from*"_", "_"*to*"_")
        @printf("mv %s %s\n", myfrom, myto)
        mv(myfrom, myto)
    end
end

change_farm_name("C12", "C13")
    

In [ ]:
res12 = lookem(farm_id="C12")

In [ ]:
gooders = find(res12[:,2].<-0.0002)

In [ ]:
# Investigate where farm C12 went outside the bounding box


bbox = load("../NewFarms/farm_C13_0001.jld", "bbox")
print(bbox)



In [ ]:
function out_of_box(pdict, bbox)
    outs = Dict()
    for k in keys(bbox)
        val = pdict[k]
        if val <= bbox[k][1] || bbox[k][2] <= val
            outs[k] = val
        end
    end
    return outs
end

for k in find(res12[:,2].<-0.0002)
    mynum = @sprintf("%d", k); while length(mynum)<4; mynum="0"*mynum; end;
    
    traj3, args = load("../NewFarms/farm_C12_" * mynum * ".jld", "traj3", "args") 
    @printf("farm_C12_%s.jld:\n", mynum)
    print(out_of_box(make_dict(args, traj3[3:end,end]), bbox))
    print("\n")
end


In [ ]:
# Sanity check on whether we are getting results in the trajectory that matches algorithm
# behavior

ngoods = [0, 0]
nguys  = [0, 0]

farm_id = "C6"; farmdir = "../NewFarms"
for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(farmdir * "/"))
    d = load(farmdir * "/" * f)
    @printf("n=%d %s: ", d["ntries"], f); 
    if d["ntries"]==1
        print(d["traj1"][2,end])        
        nguys[1] += 1
        ngoods[1] += d["traj1"][2,end] <= -0.0009 ? 1 : 0
    else
        print(d["traj3"][2,end])
        nguys[2] += 1
        ngoods[2] += d["traj3"][2,end] <= -0.0009 ? 1 : 0
    end
    @printf("  hBP=%.2f, hBA=%.2f dP=%.2f dA=%.2f", d["hBP"][1], d["hBA"][1], 
        d["dP"][1], d["dA"][1])
    @printf("\n")
end

@printf("\n For ntries=1, success rate %.2f%%, for ntries=2, rate %.2f%%\n", 100*ngoods[1]/nguys[1], 100*ngoods[2]/nguys[2])

["nguys" "ngoods" ; [nguys ngoods]]

In [ ]:
# For C6:

For ntries=1, success rate 100.00%, for ntries=2, rate 13.29%
Out[84]:
3×2 Array{Any,2}:
    "nguys"     "ngoods"
 111         111        
 143          19     
    
    
# For C7:
For ntries=1, success rate 100.00%, for ntries=2, rate 10.71%
Out[85]:
3×2 Array{Any,2}:
    "nguys"     "ngoods"
 107         107        
 140          15      

In [ ]:
farm_id = "C10"; farmdir = "../NewFarms"
for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(farmdir * "/"))
    d = load(farmdir * "/" * f)

    @printf("%s: qu_out[1]=%.2f, cost=%g\n", f, d["qu_out"][1], d["traj3"][2,end]); 
end

In [ ]:
pygui(true)
figure(1); clf();

myouts = copy(outs);
maxx   = -0.0007

myouts[myouts[:,1].>maxx,1] = maxx
myouts[myouts[:,2].>maxx,2] = maxx

ax1 = subplot(2,1,1);
plt[:hist](outs[:,1]*1000,-1:0.01:maxx*1000)
vlines(-0.95, ylim()[1], ylim()[2], color="r", linestyle="--")
ylabel("# of runs")
title("standard minimization -- left of red dashed line is a good run")
remove_xtick_labels()


ax2 = subplot(2,1,2)
plt[:hist](outs[:,2]*1000,-1:0.01:maxx*1000)
vlines(-0.95, ylim()[1], ylim()[2], color="r", linestyle="--")
xlabel("Final training cost/1000")
ylabel("# of runs")
title("minimization with pre-check")



In [ ]:
outs

In [ ]:
mypars, extra_pars, start_pars, qu_out, args, bbox = load("../NewFarms/farm_C10_0008.jld", 
    "mypars", "extra_pars", "start_pars", "qu_out", "args", "bbox")


func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=false, 
    merge(merge(mypars, extra_pars), Dict(params))...)[1]
    
            
pars3, traj3, cost3, cpm_traj3, ftraj3 = bbox_Hessian_keyword_minimization(start_pars, 
    args, bbox, func1, 
    start_eta = 0.01, tol=1e-12, 
    verbose=true, verbose_every=1, maxiter=maxiter1)



In [ ]:
        cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(10000, 10000; verbose=false, 
            make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)
        


In [ ]:
[hBP, hBA, dP, dA]'

In [ ]:
        cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(200, 200; verbose=false, 
            make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)


In [ ]:
[hBP, hBA, dP, dA]'

In [ ]:
[0.5/sqrt(200) 0.5/sqrt(400)]

In [ ]:
d = load("../NewFarms/farm_C6_0002.jld")

mypars     = d["mypars"]
extra_pars = d["extra_pars"] 
args       = d["args"]
pars1      = d["pars1"]

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(400, 400; verbose=false, 
            make_dict(args, pars1, merge(merge(mypars, extra_pars)))...)


In [ ]:
extra_pars[:seedrand]

In [ ]:
~( abs(hBP[1]-0.9)<0.075 && abs(hBA[1]-0.7)<0.075 && dP[1] > 0.8 && dA[1] > 0.8)

In [ ]:
#@include_me  analyze_reduced_farm.jl

"""
    go_over_good_runs(files, I)

Given a list of files, and a list of indices into those files, iterates over the indicated
filenames, running show_run() on them, and then asking for user input.
"""
function go_over_good_runs(files, I)
    for k=1:length(I)
        show_run(files[I[k]])

        @printf("Any key to continue, q to quit: ")
        ans = chomp(readline())
        if ans=="q" || ans=="Q"
            break
        end
    end
end


In [ ]:
go_over_good_runs(files, I)

In [ ]:
#@include_me analyze_reduced_farm.jl

# show_run("farm_C3_0027.jld", 400; opto_times=["trial_start", "trial_end"], opto_strength=0.75)
# show_run("farm_C3_0027.jld", 400; opto_times=["target_start-0.5", "target_start"], opto_strength=0, 
#   const_pro_bias=0, anti_rule_strength=0.05)
show_run("farm_C3_0027.jld", 400; opto_times=["target_start+0.016", "target_end"], opto_strength=0.4, 
    const_pro_bias=0.02476, anti_rule_strength=0.054)

In [ ]:
#@include_me analyze_reduced_farm.jl

#  TARGET PERIOD OPTO

run_name = "farm_C3_0027.jld"
recompute_me = false; if recompute_me
    ostrengths = 0.5:0.1:1
    phits      = zeros(1, length(ostrengths))
    ahits      = zeros(1, length(ostrengths))

    for i=1:length(ostrengths)
        phits[i], ahits[i] = show_run(run_name, 4000; 
        opto_times=["target_start+0.016", "target_end+0.016"], opto_strength=ostrengths[i])
        @printf("ostrength=%g, phits=%.1f, ahits=%.1f\n", ostrengths[i], phits[i], ahits[i])
    end

    @save("Temp/opto_target.jld", ostrengths, ahits, phits)
else
    ostrengths, ahits, phits = load("Temp/opto_target.jld", "ostrengths", "ahits", "phits")
end

figure(3);clf();
plot(ostrengths, phits[:], ".-", ostrengths, ahits[:], ".-")
legend(["pro", "anti"])
xlabel("opto strength")
ylabel("binarized hits")
title(run_name * " : TARGET PERIOD opto")
grid("on")


#  DELAY PERIOD OPTO  ---------

recompute_me = false; if recompute_me
    ostrengths = 0.5:0.1:1
    phits      = zeros(1, length(ostrengths))
    ahits      = zeros(1, length(ostrengths))

    for i=1:length(ostrengths)
        phits[i], ahits[i] = show_run(run_name, 4000; 
        opto_times=["target_start-0.5", "target_start"], opto_strength=ostrengths[i])
        @printf("ostrength=%g, phits=%.1f, ahits=%.1f\n", ostrengths[i], phits[i], ahits[i])
    end
    @save("Temp/opto_delay.jld", ostrengths, ahits, phits)
else
    ostrengths, ahits, phits = load("Temp/opto_delay.jld", "ostrengths", "ahits", "phits")
end

figure(4);clf();
plot(ostrengths, phits[:], ".-", ostrengths, ahits[:], ".-")
legend(["pro", "anti"])
xlabel("opto strength")
ylabel("binarized hits")
title(run_name * " : last 500 ms of DELAY PERIOD opto")
grid("on")

## workspace for double_search.jl

In [ ]:
extra_pars[:seedrand] = 1510799260433 # Int64(round(1000*time()))   # 1510782006169 causes lin.alg error but then looks like it'll succeed
srand(extra_pars[:seedrand])

    args = []; seed = []; bbox = Dict()
    for k in keys(search_conditions)
        search_box = search_conditions[k][2]
        args = [args; String(k)]
        # --- search within the indicated search range:
        myseed = search_conditions[k][1] + search_range*(rand()-0.5)*diff(search_box); myseed = myseed[1]
        if myseed > search_box[2]; myseed = search_box[2]; end
        if myseed < search_box[1]; myseed = search_box[1]; end
        seed = [seed ;  myseed]
        # --- No search, just start at the indicated position:
        # seed = [seed ; search_conditions[k][1]]
        bbox = merge(bbox, Dict(k => Array{Float64}(search_conditions[k][3])))
    end
    args = Array{String}(args)
    seed = Array{Float64}(seed)


func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=false, 
    merge(merge(mypars, extra_pars), Dict(params))...)[1]

t_pars = [0.378, 0.164, 0.001, -0.203, 0.589, -0.673, 0.105, -0.582, 0.471]

pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(t_pars, args, bbox, func1, 
    start_eta = 0.1, tol=1e-12, 
    verbose=true, verbose_every=1, maxiter=1000)


In [ ]:
testruns = 1600

pars = [0.356712, 0.000825015, 0.0218232, -0.0364268, 0.678269, -2.97884, 0.153528, -0.484789, 1.07313]

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(testruns, testruns; verbose=false, 
                make_dict(args, pars, merge(merge(mypars, extra_pars)))...)


In [ ]:
G = load("FarmFields/farm_C4_0041.jld")

In [ ]:
( abs(hBP[1]-0.9)<0.05 && abs(hBA[1]-0.7)<0.05 && dP[1] > 0.8 && dA[1] > 0.8)

In [ ]:
    args = []; seed = []; bbox = Dict()
    for k in keys(search_conditions)
        search_box = search_conditions[k][2]
        args = [args; String(k)]
        # --- search within the indicated search range:
        myseed = search_conditions[k][1] + search_range*(rand()-0.5)*diff(search_box); myseed = myseed[1]
        if myseed > search_box[2]; myseed = search_box[2]; end
        if myseed < search_box[1]; myseed = search_box[1]; end
        seed = [seed ;  myseed]
        # --- No search, just start at the indicated position:
        # seed = [seed ; search_conditions[k][1]]
        bbox = merge(bbox, Dict(k => Array{Float64}(search_conditions[k][3])))
    end
    args = Array{String}(args)
    seed = Array{Float64}(seed)


    maxiter1 = 10;  # for func1, the regular search
    maxiter2 = 20;  # for func2, the reduced search

    func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=false, 
    merge(merge(mypars, extra_pars), Dict(params))...)[1]
    
    # For func2:
    extra_pars[:opto_conditions] = []    
    extra_pars[:plot_list] = [1:10;]    
    extra_pars[:plot_condition] = 0

    func2 =  (;params...) -> new_J(10, 10; verbose=false, merge(merge(mypars, extra_pars), Dict(params))...)

    # For func2:
    # This function will get the output of new_J() at each iteration, and will return "true", stopping
    # the minimization, if hBP and hBA are both above a threshold.
    function stopping_func(;cost=0, func_out=[], ignored_extra_params...)
        costf, dP, dA, hBP, hBA = func_out
        return hBP[1]>=0.6 && hBA[1] >= 0.6
    end

        extra_pars[:plot_list] = []
        cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(100, 100; verbose=false, 
            make_dict(args, seed, merge(merge(mypars, extra_pars)))...)


In [ ]:
dA

In [ ]:
search_conditions = Dict(   # :param    default_start   search_box  bound_box
:vW     =>                   [mypars[:vW],                       [-0.5, 0.5],  [-3,   3]], 
:hW     =>                   [mypars[:hW],                       [-0.5, 0.5],  [-3,   3]],
:dW     =>                   [mypars[:dW],                       [-0.5, 0.5],  [-3,   3]],
:sW     =>                   [mypars[:sW],                       [0,    0.5],  [0,    3]],
:sigma  =>                   [0.11,                              [0.1,  0.2],  [0.1, 0.4]],
:constant_excitation      => [mypars[:constant_excitation],      [-1,     1],  [-2,   2]], 
:target_period_excitation => [mypars[:target_period_excitation], [0.001,0.5],  [0     4]],
:right_light_excitation   => [mypars[:right_light_excitation],   [0.05, 0.5],  [0.05, 4]],
:const_pro_bias           => [mypars[:const_pro_bias],           [-0.5, 0.5],  [-2,   2]],
# :opto_strength            => [mypars[:opto_strength],            [0.7, 0.99],  [0,    1]],
)



args = []; seed = []; bbox = Dict()

for k in keys(search_conditions)
    search_box = search_conditions[k][2]
    args = [args; String(k)]
    # myseed = rand()*diff(search_box) + search_box[1]; myseed = myseed[1]    
    # seed = [seed ;  myseed]    
    seed = [seed ; search_conditions[k][1]]    
    bbox = merge(bbox, Dict(k => Array{Float64}(search_conditions[k][3])))    
end
args = Array{String}(args)
seed = Array{Float64}(seed)

func =  (;params...) -> new_J(10, 10; verbose=true,  merge(merge(mypars, extra_pars), Dict(params))...)[1]

pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
            start_eta = 0.1, tol=1e-12, 
            verbose=true, verbose_every=1, maxiter=1000)


In [ ]:


mypars = Dict(
:init_add               =>          0,
:const_add              =>          0,
:noise                  =>          Any[],
:input                  =>          0,
:seedrand               =>          1510340076445,
:start_anti             =>          [-0.5, -0.5, -0.5, -0.5],
:start_pro              =>          [-0.5, -0.5, -0.5, -0.5],
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1.2],
:target_period          =>          0.3,
:target_periods         =>          [0.3],
:post_target_period     =>          0.3,
:post_target_periods    =>          [0.3],
:anti_rule_strength     =>          0.054,
:U_rest                 =>          0,
:theta                  =>          0.05,
:beta                   =>          0.5,
:g_leak                 =>          1,
:nsteps                 =>          301,
:dt                     =>          0.024,
:tau                    =>          0.09,
:right_light_excitation =>          0.49924152955481954,
:opto_strength          =>          0.85,
:opto_periods           =>          String[
                                    "trial_start" "trial_start"; 
                                    "trial_start" "trial_end"; 
                                    "trial_start" "target_start/2"; 
                                    "target_start/2" "target_start"; 
                                    "target_start" "target_end"],
:opto_targets          =>           [
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7],
:theta2 => 0.15,
:theta1 => 0.05,
:sigma => 0.01,
:cbeta => 0.04,
:sW => 0.6416875048295452,
:hW => 0.054701836208134846,
:dW => 0.1267124266934907,
:vW => -1.588850577499782,
:constant_excitation => -0.37242520737694207,
:const_pro_bias => 0.04366897857834884,
:target_period_excitation => 0.15315254453690974,
:right_light_pro_extra => 0,
:pro_rule_strength => 0.05,
:nPro => 100,
:nAnti => 100,
)


In [ ]:


extra_pars = Dict(
:plot_list        =>   [1:10;], 
:plot_condition   =>   1,
:verbose          =>   true,
:opto_conditions  =>   Array{Any}(0, 4),
)


search_conditions = Dict(   # :param    default_start   search_box  bound_box
:vW     =>                   [mypars[:vW],                       [-0.5, 0.5],  [-3,   3]], 
:hW     =>                   [mypars[:hW],                       [-0.5, 0.5],  [-3,   3]],
:dW     =>                   [mypars[:dW],                       [-0.5, 0.5],  [-3,   3]],
:sW     =>                   [mypars[:sW],                       [0,    0.5],  [0,    3]],
:sigma  =>                   [0.11,                              [0.1,  0.2],  [0.1, 0.4]],
:constant_excitation      => [mypars[:constant_excitation],      [-1,     1],  [-2,   2]], 
:target_period_excitation => [mypars[:target_period_excitation], [0.001,0.5],  [0     4]],
:right_light_excitation   => [mypars[:right_light_excitation],   [0.05, 0.5],  [0.05, 4]],
:const_pro_bias           => [mypars[:const_pro_bias],           [-0.5, 0.5],  [-2,   2]],
# :opto_strength            => [mypars[:opto_strength],            [0.7, 0.99],  [0,    1]],
)



In [ ]:
a=(1, 2, 3, 4, "5")

b,c,d,e,f = a
f


In [ ]:
#@include_me interesting_minimization.jl

include("pro_anti.jl")

In [ ]:
#@include_me interesting_minimization.jl

pygui(true)
d = load("FarmFields/farm_C3_0005.jld")
mypars     = d["mypars"]
extra_pars = d["extra_pars"] 

seed = d["seed"]
args = d["args"]
bbox = d["bbox"]

extra_pars[:opto_conditions] = []
extra_pars[:plot_list] = [1:10;]
extra_pars[:plot_condition] = 1

# This function will get the output of new_J() at each iteration, and will return "true", stopping
# the minimization, if hBP and hBA are both above a threshold.
function stopping_func(;cost=0, func_out=[], ignored_extra_params...)
    costf, dP, dA, hBP, hBA = func_out
    return hBP[1]>=0.6 && hBA[1] >= 0.6
end

func =  (;params...) -> new_J(10, 10; verbose=true, merge(merge(mypars, extra_pars), Dict(params))...)

t_pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
    stopping_function = stopping_func, 
start_eta = 0.1, tol=1e-12, verbose=true, verbose_every=1, maxiter=2000)


In [ ]:
t_pars = pars

In [ ]:
#@include_me  interesting_minimization.jl

seed = t_pars

func =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=false, 
    merge(merge(mypars, extra_pars), Dict(params))...)


pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
    start_eta = 0.1, tol=1e-12, verbose=true, verbose_every=1, maxiter=1000)


In [ ]:
README = """
For the code that generated this file, see interesting_minimization.jl.

The thing that is interesting, is that at the end of ths minimization, things are going VERY slowly, 
and cos theta is almost always close to 0! What is going on?  What is the landscape like around
there? Are we really taking nicely-directed Hessian jumps?  Or is the wall screwing that up?
"""

@save "Temp/interesting_minimization.jld"


In [ ]:
gu = true

@printf("gu is %s\n", false)

In [ ]:



fbasename = "FarmFields/farm_C4_"

# while true
    args = []; seed = []; bbox = Dict()
    for k in keys(search_conditions)
        search_box = search_conditions[k][2]
        args = [args; String(k)]
        myseed = rand()*diff(search_box) + search_box[1]; myseed = myseed[1]
        seed = [seed ;  myseed]
        # seed = [seed ; search_conditions[k][1]]
        bbox = merge(bbox, Dict(k => Array{Float64}(search_conditions[k][3])))
    end
    args = Array{String}(args)
    seed = Array{Float64}(seed)



    func =  (;params...) -> new_J(10, 10; verbose=true, 
        merge(merge(mypars, extra_pars), Dict(params))...)[1]

    try
        pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
            start_eta = 0.1, tol=1e-12, 
            verbose=true, verbose_every=1, maxiter=1000)


        cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(10000, 10000; verbose=false, 
        make_dict(args, pars, merge(merge(mypars, extra_pars)))...)

        myfilename = next_file(fbasename, 4)
        myfilename = myfilename*".jld"
        # write file
        save(myfilename, Dict("nPro"=>mypars[:nPro], "nAnti"=>mypars[:nAnti], 
        "mypars"=>mypars, "extra_pars"=>extra_pars, "args"=>args, "seed"=>seed, "bbox"=>bbox, 
        "pars"=>pars, "traj"=>traj, "cost"=>cost, "cpm_traj"=>cpm_traj, "ftraj"=>ftraj,
        "hP"=>hP, "hA"=>hA, "dP"=>dP, "dA"=>dA, "hBP"=>hBP, "hBA"=>hBA))
    catch
        @printf("\n\nWhoopsety, unkown error!   Trying new random seed.\n\n")
    end
# end

# Alex-style runs

In [ ]:
using ForwardDiff
using DiffBase
using MAT
using PyPlot


In [ ]:
include("pro_anti.jl")

In [ ]:
FarmName = "CARLOS_TEST"


#Define core model parameters
model_params = Dict(
:dt     =>  0.002,      # Changed hyper-parameter
:tau    =>  0.02,       # Changed hyper-parameter
:vW     =>  -1.7, #FIT parameter
:hW     =>  -1.7, #FIT parameter
:sW     =>  0.2,  #FIT parameter
:dW     =>  0,    #FIT parameter
:nsteps =>  2, 
:noise  =>  [], 
:sigma  =>  0.08, #FIT parameter
:input  =>  0,    #### Alex thinks this is out-dated 
:g_leak =>  1,          # Changed hyper-parameter
:U_rest =>  0,          # Changed hyper-parameter
:theta  =>  0.05,       # Changed hyper-parameter
:beta   =>  0.5,        # Changed hyper-parameter 
:sw     =>  0.2,  #### Alex thinks this is out-dated
:hw     =>  -1.7, #### Alex thinks this is out-dated
:vw     =>  -1.7, #### Alex thinks this is out-dated
:constant_excitation      => 0.19, #FIT  parameter
:anti_rule_strength       => 0.05,      # Changed hyper-parameter
:pro_rule_strength        => 0.05,      # Changed hyper-parameter
:target_period_excitation => 1,    #FIT  parameter
:right_light_excitation   => 0.5,  #FIT parameter
:right_light_pro_extra    => 0,
:const_add                => 0, 
:init_add                 => 0, 
:rule_and_delay_period    => 0.4,
:target_period            => 0.1,
:post_target_period       => 0.5,
:const_pro_bias           => 0,    #FIT parameter
:nPro                     => 100,
:nAnti                    => 100,
:theta1                   => 0.01,
:theta2                   => 0.15,
:start_pro                => [-0.5, -0.5, -0.5, -0.5],
:start_anti               => [-0.5, -0.5, -0.5, -0.5],
:opto_strength  => .7,             #FIT parameter
:opto_periods   => [-1.1 -1; 0 20;0 50; 50 100; 100 20],  
# The opto "conditions" correspond to the rows of opto_periods.
# all conditions are in seconds relative to start of the trial
# any value before 0 gets changed to 0
# any value after the end of trial gets changed to end of trial
# Special values allow for variable durations
#  20 codes for "end of trial"
#  -1 codes for "start of trial"
# 100 codes for "end of rule and delay period"
# 200 codes for "end of target period"
# -------------------------------------------------------------
# first column is frachit Pro, next column is Anti, rows are conditions
# Actual Opto targets
#:opto_targets   => [.75 .73;.77 .58;.75 .74; .72 .66;.73 .75] 
# Fake Targets
:opto_targets => [.9 .7; .9 .5; .9 .7; .9 .5; .9 .7]  
);

# ======= ARGUMENTS AND SEED VALUES:
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation", "const_pro_bias", "sigma","opto_strength"];
seed = [0.001,  -1.58,   -0.05,  0.001,    0.001,                0.6,                       0.001,                        0.0427,              0.05, .9];   
# ======= BOUNDING BOX:
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0 4], :const_pro_bias=>[-2 2],
:sigma=>[0.1 0.4],:opto_strength=>[0 1]);

# ======== SEARCH ZONE:
sbox = Dict(:sW=>[0 .5], :vW=>[-.5 .5], :hW=>[-.5 .5], :dW=>[-.5 .5],
:constant_excitation=>[-.5 .5], :right_light_excitation=>[0.05 .5], :target_period_excitation=>[0.001 .5],:const_pro_bias=>[-.5 .5], :sigma=>[0.1 .2],:opto_strength=>[.7 .99]);

# define a few hyper parameters
cbetas = [4e-3];
rule_and_delay_periods = [0.2];
post_target_periods    = [0.05];
num_eval_runs           = 1000;
num_optimize_iter       = 2000;
num_optimize_restarts   = 100;

# define base filename
fbasename = "FarmFields/farm_"*string(FarmName);

for i=1:num_optimize_restarts   # Iterate this many optimizations
for cb in cbetas                # Iterate over beta values, if there are multiple
    # figure out initial seed for random number generator
    sr = convert(Int64, round(time()))
    srand(sr);

    dista=rand();
    println("dista")
    println(dista)
    myseed1 = copy(seed);

    # get initial parameter values by sampling from sbox
    myseed = ForwardDiffZeros(length(args),1);
    for j=1:length(args)
        sym = Symbol(args[j])
        if haskey(sbox, sym)
#           myseed[j] = sbox[sym][1] + diff(sbox[sym],2)[1]*rand();
            myseed[j] = dista*(sbox[sym][1] + diff(sbox[sym],2)[1]*rand())+(1-dista)*myseed1[j];
        if myseed[j] > bbox[sym][2]
            myseed[j] =bbox[sym][2] - 0.1;
        elseif myseed[j] < bbox[sym][1]
            myseed[j] =bbox[sym][1]+0.1;
        end
        else
            myseed[j] = seed[j];
        end
    end

    # define opto function with just value output
    func =  (;params...) -> JJ(model_params[:nPro], model_params[:nAnti]; 
        rule_and_delay_periods=rule_and_delay_periods, 
        theta1=model_params[:theta1], theta2=model_params[:theta2], 
        post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, 
        verbose=false, merge(model_params, Dict(params))...)[1]
    
    # run optimization with all parameters
    @printf("Going with seed = "); print_vector_g(myseed); print("\n")
    pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(myseed, args, bbox, func, start_eta = 1, tol=1e-12, verbose=true, verbose_every=10, maxiter=num_optimize_iter)
    @printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")

    # get gradient and hessian at end of optimization 
    value, grad, hess = keyword_vgh(func, args, pars)

    # define function with all outputs, evaluate on training noise
    t_standard_func =  (;params...) -> JJ_opto(model_params[:nPro], model_params[:nAnti]; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run opto model with all outputs, evaluate on training noise
    t_opto_scost, t_opto_scost1, t_opto_scost2, t_opto_hitsP,t_opto_hitsA, t_opto_diffsP, t_opto_diffsA, t_opto_bP, t_opto_bA = t_standard_func(;make_dict(args, pars, model_params)...)
    
   ## evaluate for long form info
    # reset random number generator for testing purposes
    test_sr = convert(Int64, round(time()))
    srand(test_sr); 

    # define function with all outputs, evaluate on test noise
    standard_func =  (;params...) -> JJ_opto(num_eval_runs, num_eval_runs; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=test_sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run opto model with all outputs, evaluate on test noise
    opto_scost, opto_scost1, opto_scost2, opto_hitsP,opto_hitsA, opto_diffsP, opto_diffsA, opto_bP, opto_bA = standard_func(;make_dict(args, pars, model_params)...)

   ## Save this run out to a file
    # get filename
    myfilename = next_file(fbasename, 4)
    myfilename = myfilename*".mat"
    # write file
    matwrite(myfilename, Dict("args"=>args, "myseed"=>myseed, "pars"=>pars, "traj"=>traj, "cost"=>cost, "cpm_traj"=>cpm_traj, "nPro"=>model_params[:nPro], "nAnti"=>model_params[:nAnti], "sr"=>sr, "cb"=>cb, "theta1"=>model_params[:theta1], "theta2"=>model_params[:theta2],"value"=>value,"grad"=>grad, "hess"=>hess, "model_params"=>ascii_key_ize(model_params), "bbox"=>ascii_key_ize(bbox), "sbox"=>ascii_key_ize(sbox), "rule_and_delay_periods"=>rule_and_delay_periods, "post_target_periods"=>post_target_periods, "opto_scost"=>opto_scost, "opto_scost1"=>opto_scost1, "opto_scost2"=>opto_scost2, "opto_hitsP"=>opto_hitsP, "opto_hitsA"=>opto_hitsA, "opto_diffsP"=>opto_diffsP, "opto_diffsA"=>opto_diffsA,"test_sr"=>test_sr,"opto_bP"=>opto_bP, "opto_bA"=>opto_bA, "t_opto_scost"=>t_opto_scost, "t_opto_scost1"=>t_opto_scost1, "t_opto_scost2"=>t_opto_scost2, "t_opto_hitsP"=>t_opto_hitsP, "t_opto_hitsA"=>t_opto_hitsA, "t_opto_diffsP"=>t_opto_diffsP, "t_opto_diffsA"=>t_opto_diffsA,"t_opto_bP"=>t_opto_bP, "t_opto_bA"=>t_opto_bA,"dista"=>dista  ))
end
end



In [ ]:
using HypothesisTests

In [ ]:
N = 20
x = 1+0.2*randn(N,1); y = x - 0.05 + 0.1*randn(N,1);


using PyPlot
pygui(true)

figure(1); clf();
subplot(1,2,1)

for i=1:N
    plot([1, 2], [x[i], y[i]], "b.-")
end

p = pvalue(OneSampleTTest(x[:], y[:]))

title("Paired t-test :  p<" * @sprintf("%.4g", p))


subplot(1,2,2)
plot(x-y, "b.")

# Marino's model in Marino's parameter settings

In [ ]:
#@include_me  scaling_up_dt.jl

# In its own cell so we can run it just once

include("pro_anti.jl")   # Loads all the necessary pre-requisites


In [ ]:
#@include_me  scaling_up_dt.jl

model_params, F, nPro, nAnti = load_run("farm_LD0003"; farmdir="goodfarms")

# mypars = merge(model_params, Dict(:post_target_periods=>0.05))

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA, proValls, antiValls, opto_fraction, pro_input, anti_input = 
JJ(model_params[:nPro], model_params[:nAnti]; verbose=true, model_details=true, model_params...);


epochs = ["control";"full";"rule";"delay";"choice"]
titles  = ["epochs" "hitP" "hitA" "diffP" "diffA" "hBP" "hBA"]

@printf("\n\n--- The original run:\n\n")

sleep(0.5)  # A pause just to make sure everything has printed out 
display([titles; epochs hP hA dP dA hBP hBA])


In [ ]:
#@include_me  scaling_up_dt.jl

# Now scale things to make dt and tau are much larger, and involve 0.25 x the timesteps

mypars = merge(model_params, Dict(:opto_times=>[0 0]))  # default is no opto

mypars = merge(model_params, Dict(
:dt=>0.024,
:tau=>0.09,
:rule_and_delay_period=>1.2,
:target_period=>0.3,
:post_target_period=>0.3,
:anti_rule_strength=>0.054,
# :post_target_period=>0.05, 
# :rule_and_delay_period=>0.8, 
# :opto_times=>[0 0],
# :opto_times=>["target_start/2" "target_start"],
# :opto_times=>["target_start", "target_end"],
:opto_strength=>0.85,
))# , :rule_and_delay_period=>0.8))


mypars[:rule_and_delay_periods] = [mypars[:rule_and_delay_period]]
mypars[:target_periods]         = [mypars[:target_period]]
mypars[:post_target_periods]    = [mypars[:post_target_period]]

mypars[:opto_periods] = [
    "trial_start"     "trial_start"  ;
    "trial_start"     "trial_end"    ; 
    "trial_start"     "target_start/2"  ;
    "target_start/2"  "target_start"  ;
    "target_start"    "target_end"
];


In [ ]:
#@include_me  scaling_up_dt.jl

# And run with these params.  

# We run 1000 trials of Pro and ANti each, just to get a good estimate of probability correct in each condition

mypars[:seedrand] = Int64(round(time()*1000))

ntrials = 1000

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA, proValls, antiValls, opto_fraction, pro_input, anti_input = 
JJ(ntrials, ntrials; verbose=true, model_details=true, mypars...);

@printf("\n\n--- The hand-modified run at dt=0.024 at 4x fewer timesteps than the original:\n\n")


sleep(0.5)  # A pause just to make sure everything has printed out 
display([titles; epochs hP hA dP dA hBP hBA])


In [ ]:
#@include_me  scaling_up_dt.jl

# Finally, display some trajectories:

pygui(true)

# This example is with opto during the delay

mypars = merge(mypars, Dict(
# :opto_times => ["target_start/2"    "target_start"],
:opto_times => [0    0],
:sigma=>0.01
))


proVs, antiVs, pro_fullV, anti_fullV, opto_fraction, pro_input, anti_input = run_ntrials(1000, 1000; 
    plot_list=[1:40;], profig=1, antifig=2, opto_units = 1:4, mypars...);

Np = size(proVs,2); Na = size(antiVs,2)
@printf("hBP = %g, hBA = %g\n", length(find(proVs[1,:] .>= proVs[4,:]))/Np, length(find(antiVs[4,:] .> antiVs[1,:]))/Na)

@printf("\n\n\n---\n\nmypars contains all the parameters for the new run with the scaled up dt, tau, etc.\n\n")


In [ ]:
mypars[:dt]

In [ ]:
# Print out mypars in full
@printf("\n\nmypars = Dict(\n")
for (k,v) in mypars
    @printf(":%s => ", k); print(v); print(",\n")
end
@printf(")\n\n")

# Make a set of reduced nsteps params and run with them

In [ ]:

mypars = Dict(
:init_add               =>          0,
:const_add              =>          0,
:noise                  =>          Any[],
:input                  =>          0,
:seedrand               =>          1510340076445,
:start_anti             =>          [-0.5, -0.5, -0.5, -0.5],
:start_pro              =>          [-0.5, -0.5, -0.5, -0.5],
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1.2],
:target_period          =>          0.3,
:target_periods         =>          [0.3],
:post_target_period     =>          0.3,
:post_target_periods    =>          [0.3],
:anti_rule_strength     =>          0.054,
:U_rest                 =>          0,
:theta                  =>          0.05,
:beta                   =>          0.5,
:g_leak                 =>          1,
:nsteps                 =>          301,
:dt                     =>          0.024,
:tau                    =>          0.09,
:right_light_excitation =>          0.49924152955481954,
:opto_strength          =>          0.85,
:opto_periods           =>          String[
                                    "trial_start" "trial_start"; 
                                    "trial_start" "trial_end"; 
                                    "trial_start" "target_start/2"; 
                                    "target_start/2" "target_start"; 
                                    "target_start" "target_end"],
:opto_targets          =>           [
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7],
:theta2 => 0.15,
:theta1 => 0.05,
:sigma => 0.01,
:cbeta => 0.04,
:sW => 0.6416875048295452,
:hW => 0.054701836208134846,
:dW => 0.1267124266934907,
:vW => -1.588850577499782,
:constant_excitation => -0.37242520737694207,
:const_pro_bias => 0.04366897857834884,
:target_period_excitation => 0.15315254453690974,
:right_light_pro_extra => 0,
:pro_rule_strength => 0.05,
:nPro => 100,
:nAnti => 100,
)


In [ ]:

extra_pars = Dict(
:plot_list        =>   [1:5;], 
:plot_conditions  =>   true,
:verbose          =>   true,
:opto_periods     =>   String["trial_start" "trial_start"],
:opto_targets     =>   [0.9,  0.7],
:opto_times       =>   ["trial_start", "trial_start"],       # This one is for run_ntrials
:seedrand         =>   Int64(round(time()*1000)),
)


# proVs, antiVs, pro_fullV, anti_fullV, opto_fraction, pro_input, anti_input = run_ntrials(1000, 1000; 
#    merge(mypars, extra_pars)...);


JJ(mypars[:nPro], mypars[:nAnti]; merge(mypars, extra_pars)...)[1]


In [ ]:
extra_pars[:search_range] = 0.4
extra_pars

# Try a search : reduced_farm.jl

`Temp/answers.jld`   contains the result of taking the result of scaling_up_dt.jl, and then optimizing it on control alone, with Pro target=0.9, Anti target = 0.7.  It optimizes very very well.

In [ ]:
#@include_me reduced_farm.jl

include("pro_anti.jl")



In [ ]:
@load("FarmFields/farm_C3_0027.jld")

In [ ]:
[args pars]

In [ ]:
mypars[:tau]

In [ ]:
#@include_me reduced_farm.jl


mypars = Dict(
:init_add               =>          0,
:const_add              =>          0,
:noise                  =>          Any[],
:input                  =>          0,
:start_anti             =>          [-0.5, -0.5, -0.5, -0.5],
:start_pro              =>          [-0.5, -0.5, -0.5, -0.5],
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1.2],
:target_period          =>          0.3,
:target_periods         =>          [0.3],
:post_target_period     =>          0.3,
:post_target_periods    =>          [0.3],
:anti_rule_strength     =>          0.054,
:U_rest                 =>          0,
:theta                  =>          0.05,
:beta                   =>          0.5,
:g_leak                 =>          1,
:nsteps                 =>          301,
:dt                     =>          0.024,
:tau                    =>          0.09,
:right_light_excitation =>          0.49924152955481954,
:opto_strength          =>          0.85,
:opto_periods           =>          String[
                                    "trial_start" "trial_start"; 
                                    "trial_start" "trial_end"; 
                                    "trial_start" "target_start/2"; 
                                    "target_start/2" "target_start"; 
                                    "target_start" "target_end"],
:opto_targets          =>           [
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7],
:theta2 => 0.15,
:theta1 => 0.05,
:sigma => 0.01,
:cbeta => 0.04,
:sW => 0.6416875048295452,
:hW => 0.054701836208134846,
:dW => 0.1267124266934907,
:vW => -1.588850577499782,
:constant_excitation => -0.37242520737694207,
:const_pro_bias => 0.04366897857834884,
:target_period_excitation => 0.15315254453690974,
:right_light_pro_extra => 0,
:pro_rule_strength => 0.05,
:nPro => 100,
:nAnti => 100,
)


In [ ]:
#@include_me  reduced_farm.jl


extra_pars = Dict(
:plot_list        =>   [], 
:plot_conditions  =>   true,
:verbose          =>   true,
:opto_periods     =>   String["trial_start" "trial_start-0.1"],
:opto_targets     =>   [0.9,  0.7],
:opto_times       =>   ["trial_start", "trial_start-0.1"],       # This one is for run_ntrials
:cbeta            =>   0.001,
:search_range     =>   1.2,
)


search_conditions = Dict(   # :param    default_start   search_box  bound_box
:vW     =>                   [mypars[:vW],                       [-0.5, 0.5],  [-3,   3]], 
:hW     =>                   [mypars[:hW],                       [-0.5, 0.5],  [-3,   3]],
:dW     =>                   [mypars[:dW],                       [-0.5, 0.5],  [-3,   3]],
:sW     =>                   [mypars[:sW],                       [0,    0.5],  [0,    3]],
:sigma  =>                   [0.11,                              [0.1,  0.2],  [0.1, 0.4]],
:constant_excitation      => [mypars[:constant_excitation],      [-1,     1],  [-2,   2]], 
:target_period_excitation => [mypars[:target_period_excitation], [0.001,0.5],  [0     4]],
:right_light_excitation   => [mypars[:right_light_excitation],   [0.05, 0.5],  [0.05, 4]],
:const_pro_bias           => [mypars[:const_pro_bias],           [-0.5, 0.5],  [-2,   2]],
# :opto_strength            => [mypars[:opto_strength],            [0.7, 0.99],  [0,    1]],
)




### Loop for reduced_farm.jl

In [ ]:
#@include_me  reduced_farm.jl

extra_pars[:seedrand] = Int64(round(time()*1000))
srand(extra_pars[:seedrand])

@printf("\n\n\nStarting with random seed %d\n\n\n", extra_pars[:seedrand])

search_range = extra_pars[:search_range]; 

fbasename = "FarmFields/farm_C5_"

while true
    args = []; seed = []; bbox = Dict()
    for k in keys(search_conditions)
        search_box = search_conditions[k][2]
        args = [args; String(k)]
        myseed = search_conditions[k][1] + search_range*(rand()-0.5)*diff(search_box); myseed = myseed[1]
        if myseed > search_box[2]; myseed = search_box[2]; end
        if myseed < search_box[1]; myseed = search_box[1]; end
        seed = [seed ;  myseed]
        # seed = [seed ; search_conditions[k][1]]
        bbox = merge(bbox, Dict(k => Array{Float64}(search_conditions[k][3])))
    end
    args = Array{String}(args)
    seed = Array{Float64}(seed)


    # Make sure to keep the noise frozen over the search, meaning JJ() needs the seedrand parameter
    func =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=true, 
        merge(merge(mypars, extra_pars), Dict(params))...)[1]

    try
        pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
            start_eta = 0.1, tol=1e-12, 
            verbose=true, verbose_every=1, maxiter=1000)


        cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(1000, 1000; verbose=false, 
        make_dict(args, pars, merge(merge(mypars, extra_pars)))...)

        myfilename = next_file(fbasename, 4)
        myfilename = myfilename*".jld"
        # write file
        save(myfilename, Dict("nPro"=>mypars[:nPro], "nAnti"=>mypars[:nAnti], 
        "mypars"=>mypars, "extra_pars"=>extra_pars, "args"=>args, "seed"=>seed, "bbox"=>bbox, 
        "pars"=>pars, "traj"=>traj, "cost"=>cost, "cpm_traj"=>cpm_traj, "ftraj"=>ftraj,
        "hP"=>hP, "hA"=>hA, "dP"=>dP, "dA"=>dA, "hBP"=>hBP, "hBA"=>hBA))
    catch y
        if isa(y, InterruptException); throw(InterruptException()); end
        @printf("\n\nWhoopsety, unkown error!\n\n");
        @printf("Error was :\n"); print(y); @printf("\n\nTrying new random seed.\n\n")
    end

    # Change the frozen noise random seed so we don't get stuck in one loop    
    extra_pars[:seedrand] = extra_pars[:seedrand]+1
end

In [ ]:
effective_search_box = Dict()
for k in keys(search_conditions)
    search_box = search_conditions[k][2]
    ebox = zeros(1,2)
    ebox[1] = search_conditions[k][1] - 0.5*diff(search_box)[1]
    ebox[2] = search_conditions[k][1] + 0.5*diff(search_box)[1]
    effective_search_box[k] = ebox
end

effective_search_box

# Analyze a Marino goodfarm:

In [ ]:
include("pro_anti.jl")

In [ ]:
hBP, hBA, d = show_run("farm_LA0001.mat", 100; farmdir="goodfarms", opto_times=["target_start-0.2", "target_start"], 
const_pro_bias = 0.0, opto_strength = 0)

In [ ]:
model_params = symbol_key_ize(d["model_params"])
[d["args"] d["pars"]]

In [ ]:
show_run("farm_LD0053.mat", 100; farmdir="goodfarms", opto_times=["target_start-2", "target_end-2"],
    const_pro_bias=0.04745, rule_and_delay_period=0.4, right_light_excitation=0.4)


In [ ]:
model_params, F, nPro, nAnti = load_run("farm_LD0053.mat"; farmdir="goodfarms")

In [ ]:
[F["args"] F["pars"]]

In [ ]:
model_params[:opto_periods] = [
    "trial_start-0.1"   "trial_start-0.1"; 
    "trial_start"       "trial_end" ; 
    "trial_start"       "trial_start+0.2" ; 
    "target_start-0.2"  "target_start"  ;
    "target_start"      "trial_end"
]

model_params[:opto_targets]



In [ ]:
pygui(true)
cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA, proValls, antiValls = JJ(100, 100; plot_list=[1:10;], 
    plot_conditions=[false, false, false, true, false], verbose=true, model_details=true, model_params...);

In [ ]:
include("farm_plotting_carlos.jl")

In [ ]:
show_run("farm_C3_0002.jld")


In [ ]:
figure(1); subplot(3,1,1); title(@sprintf("%s: binarized Pro  hits = %.1f %%\n", run_name, 97.3))


In [ ]:
# @save "Temp/answers.jld" 
# @load "Temp/answers.jld"

In [ ]:
@load "Temp/answers3.jld"

In [ ]:
[args pars seed]

In [ ]:
pygui(true)
figure(1); clf();

delete!(extra_pars, :plot_list)
delete!(extra_pars, :seedrand)

more_pars = Dict(
# :post_target_period => 0.25,
:rule_and_delay_period => 2,
)
proVs, antiVs = run_ntrials(1000, 1000; plot_list=[1:10;], make_dict(args, pars, merge(merge(mypars, extra_pars), more_pars))...)[1:2]

@printf("Binarized Pro hits = %.1f %%\n", 100*length(find(proVs[1,:] .> proVs[4,:]))/size(proVs,2))
@printf("Binarized Anti hits = %.1f %%\n", 100*length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2))

In [ ]:
search_conditions

In [ ]:
include("pro_anti.jl")

In [ ]:
bbox[:dW]

In [ ]:
[1, [2,3], 4]


In [ ]:
#  size(proValls)  (4, 301, 100, 5)

figure(3); clf(); N=40

t = 0:size(proValls, 2)-1; t = model_params[:dt]*t

figure(3); clf();
for i=1:N 
    plot_PA(t, 0, proValls[:,:,i,1], fignum=3, plot_Us=false,
    rule_and_delay_period=model_params[:rule_and_delay_period], target_period=model_params[:target_period],
    post_target_period = model_params[:post_target_period])
end

figure(4); clf();
for i=1:N 
    plot_PA(t, 0, antiValls[:,:,i,1], fignum=4, plot_Us=false,
    rule_and_delay_period=model_params[:rule_and_delay_period], target_period=model_params[:target_period],
    post_target_period = model_params[:post_target_period])
end


In [ ]:
print(model_params)

In [ ]:
# Define core model parameters
model_params = Dict(
:dt     =>  0.002, 
:tau    =>  0.02, 
:vW     =>  -1.58,
:hW     =>  -0.05,
:sW     =>  0,
:dW     =>  0,
:nsteps =>  301, 
:noise  =>  [], 
:sigma  =>  0.08, 
:input  =>  0, 
:g_leak =>  1, 
:U_rest =>  0,
:theta  =>  0.05, 
:beta   =>  0.5, 
:constant_excitation      => 0, 
:anti_rule_strength       => 0.05,
:pro_rule_strength        => 0.05, 
:target_period_excitation => 0,
:right_light_excitation   => 0.6, 
:right_light_pro_extra    => 0,
:const_add                => 0, 
:init_add                 => 0, 
:rule_and_delay_period    => 0.2,
:target_period            => 0.1,
:post_target_period       => 0.1,
:const_pro_bias           => 0.0427,
:nPro                     => 100,
:nAnti                    => 100,
:theta1                   => 0.05,
:theta2                   => 0.15,
:opto_strength  => .9,
:opto_periods   => [
    0               0 ; 
    0              "trial_end" ;
    "target_start/2"  "target_start";
    "target_start"    "target_end"],  
# set of opto conditions, in seconds, with 0 the start 
# of the trial (i.e. start of rule_and_delay_period), anything before 0 or after end of trial gets ignored.
#:opto_targets   => [.75 .73;.77 .58;.75 .74;.72 .66;.73 .75] 
:opto_targets => [
    .9      .7; 
    .9      .5; 
    .9      .55; 
    .9      .7;
    ],  # first column is frachit Pro, next column is Anti, rows are conditions
# The "conditions" correspond to the rows of opto_periods.
);


# ======= ARGUMENTS AND SEED VALUES:
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation"]
seed = [0.001,  -1.58,   -0.05,  0.001,    0.001,                0.6,                       0.001]   
args = [args ; ["const_pro_bias", "sigma","opto_strength", "pro_rule_strength", "anti_rule_strength"]];
seed = [seed ; [  0.0427,            .05,      .9,              0.05,                0.05]];

# ======= BOUNDING BOX:
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.2],:opto_strength=>[0 1]);

# ======== SEARCH ZONE:
sbox = Dict(:sW=>[0 .5], :vW=>[-.5 .5], :hW=>[-.5 .5], :dW=>[-.5 .5],
:constant_excitation=>[-.5 .5], :right_light_excitation=>[0.05 .5], :target_period_excitation=>[0.001 .5],:const_pro_bias=>[-.5 .5], :sigma=>[0.02 0.19],:opto_strength=>[.7 .99]);

# define a few hyper parameters
cbetas = [0.04];
rule_and_delay_periods = [0.2];
post_target_periods    = [0.1];
num_eval_runs           = 100;
num_optimize_iter       = 2000;
num_optimize_restarts   = 1;


cb = 0.04

# figure out initial seed for random number generator
sr = convert(Int64, round(time()))
srand(sr);

mypars = merge(model_params, Dict(:opto_times => ["target_start", "target_end"], :rule_and_delay_period=>0.2,
:anti_rule_strength=>0.06))
# mypars = merge(model_params, Dict(:opto_times => [0 0], :rule_and_delay_period=>0.1,
# :anti_rule_strength=>0.06))
pygui(true)
proVs, antiVs, pro_fullV, anti_fullV, opto_fraction, pro_input, anti_input = run_ntrials(15, 15; 
plot_list=[1:15;], profig=1, antifig=2, opto_units = 1:4, mypars...)

# define opto function with just value output
func =  (;params...) -> JJ(model_params[:nPro], model_params[:nAnti]; 
    rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], 
    post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, verbose=true, 
    merge(mypars, Dict(params))...)[1]


# cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(model_params[:nPro], model_params[:nAnti]; 
#     rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], 
#     post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, verbose=true, model_params...)
    

In [ ]:
@printf("Going with seed = "); print_vector_g(seed); print("\n")
pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
    start_eta = 1, tol=1e-12, verbose=true, verbose_every=1, maxiter=num_optimize_iter)
@printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")


In [ ]:
# run optimization with all parameters
@printf("Going with seed = "); print_vector_g(seed); print("\n")
pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
    start_eta = 1, tol=1e-12, verbose=true, verbose_every=1, maxiter=num_optimize_iter)
@printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")

    # get gradient and hessian at end of optimization 
    value, grad, hess = keyword_vgh(func, args, pars)

    # define function with all outputs, evaluate on training noise
    t_standard_func =  (;params...) -> JJ_opto(model_params[:nPro], model_params[:nAnti]; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run opto model with all outputs, evaluate on training noise
    t_opto_scost, t_opto_scost1, t_opto_scost2, t_opto_hitsP,t_opto_hitsA, t_opto_diffsP, t_opto_diffsA, t_opto_bP, t_opto_bA = t_standard_func(;make_dict(args, pars, model_params)...)
    
   ## evaluate for long form info
    # reset random number generator for testing purposes
    test_sr = convert(Int64, round(time()))
    srand(test_sr); 

    # define function with all outputs, evaluate on test noise
    standard_func =  (;params...) -> JJ_opto(num_eval_runs, num_eval_runs; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=test_sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run opto model with all outputs, evaluate on test noise
    opto_scost, opto_scost1, opto_scost2, opto_hitsP,opto_hitsA, opto_diffsP, opto_diffsA, opto_bP, opto_bA = standard_func(;make_dict(args, pars, model_params)...)

    # define non-opto model with all outputs, to check opto model, evaluate on test noise
    standard_func =  (;params...) -> JJ(num_eval_runs, num_eval_runs; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=test_sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run non-opto model with all outputs, evaluate on test noise
    scost, scost1, scost2, hitsP,hitsA, diffsP, diffsA = standard_func(;make_dict(args, pars, model_params)...)
 
   ## Save this run out to a file
    # get filename
    myfilename = next_file(fbasename, 4)
    myfilename = myfilename*".mat"
    # write file
    matwrite(myfilename, Dict("args"=>args, "myseed"=>myseed, "dista"=>dista, "pars"=>pars, "traj"=>traj, "cost"=>cost, "cpm_traj"=>cpm_traj, "nPro"=>model_params[:nPro], "nAnti"=>model_params[:nAnti], "sr"=>sr, "cb"=>cb, "theta1"=>model_params[:theta1], "theta2"=>model_params[:theta2],"value"=>value,"grad"=>grad, "hess"=>hess, "scost"=>scost, "scost1"=>scost1, "scost2"=>scost2, "hitsP"=>hitsP,"hitsA"=>hitsA, "diffsP"=>diffsP, "diffsA"=>diffsA, "model_params"=>ascii_key_ize(model_params), "bbox"=>ascii_key_ize(bbox), "sbox"=>ascii_key_ize(sbox), "rule_and_delay_periods"=>rule_and_delay_periods, "post_target_periods"=>post_target_periods, "opto_scost"=>opto_scost, "opto_scost1"=>opto_scost1, "opto_scost2"=>opto_scost2, "opto_hitsP"=>opto_hitsP, "opto_hitsA"=>opto_hitsA, "opto_diffsP"=>opto_diffsP, "opto_diffsA"=>opto_diffsA,"test_sr"=>test_sr,"opto_bP"=>opto_bP, "opto_bA"=>opto_bA, "t_opto_scost"=>t_opto_scost, "t_opto_scost1"=>t_opto_scost1, "t_opto_scost2"=>t_opto_scost2, "t_opto_hitsP"=>t_opto_hitsP, "t_opto_hitsA"=>t_opto_hitsA, "t_opto_diffsP"=>t_opto_diffsP, "t_opto_diffsA"=>t_opto_diffsA,"t_opto_bP"=>t_opto_bP, "t_opto_bA"=>t_opto_bA  ))
end
end